In [1]:
# -*- coding: latin1 -*-
# cython: language_level=3

In [2]:
#import
import geopandas as gpd
import os, sys
from datetime import datetime, timedelta, date
import numpy as np
import json, feather
import pandas as pd
from scipy.spatial.distance import cdist
import pyproj
import cython

We can use the following programs to obtain data from either lookup csv files (for example, for elevation data) or to obtain the data from feather files. We are using feather files for faster reading. Cython is implemented only for certain functions that will need to be sped up. 

In [3]:
#Get the index of col_name (a str) in the header of the file that file_path points to. 
def get_col_num_list(file_path,col_name): 
    '''Get the column in the look up file that corresponds to the correct data
    Parameters: 
        file_path (str): path to the lookup csv file, includes the file name
        col_name (str): the column name of the data you want to use in the lookup file, ex. "ELEV"
    Returns 
        idx_list (list): list containing all columns in the lookup table that are called col_name
    '''
    col_nums = [] #Create empty list for the relevent column numbers.
    header = []
    count = 0
    with open(file_path) as lookup_info:
        for line in lookup_info:
            row = line.rstrip('\n').split(',')
            if count == 0: 
                header.append(row[0:]) #Access the header 
            count += 1
    
    #What is the index of col_name in the header? 
    idx_list = [i for i, x in enumerate(header[0]) if col_name in x.lower()] 

    lookup_info.close() 

    return idx_list

In [4]:
#Get data from a lookup file containing the data + the lat and lon
def finding_data_frm_lookup(projected_coordinates_list,file_path,idx_list):
    '''Get the dictionary of values for each input point frm lookup file
    Parameters
        projected_coordinates_list (list of tuples): list of coordinates that you want to find data
        in the lookup file for, must be in lon, lat format 
        file_path (str): file path to the lookup file, includes the file name and ending
        idx_list (list): output of get_col_name_list, we will just take the first entry
    Returns 
        min_distance_L (dict): a dictionary of values for each coordinate (taken from the closest 
        point in the lookup file to the input coordinate)
    '''
    distance_dictionary = {} 
    L_dictionary = {}
    min_distance_L = {}
    temp = []
    with open(file_path) as L_information:
        next(L_information) # Skip header. 
        for line in L_information:
            line2 = line.rstrip('\n').split(',')
            #Store information in Python dictionary so we can access it efficiently. 
            L_dictionary[tuple([float(line2[1]),float(line2[2])])] = float(line2[idx_list[0]])
            temp.append([float(line2[1]),float(line2[2])]) #Temp lat lon list 

    L_information.close() 

    L_locarray = np.array(temp) #Make an array of the lat lons. 

    loc_dict = {} 

    for coord in projected_coordinates_list: # projected_coordinates_list --> list of tuples.
        ts_lat = coord[1] # Obtain the lat/lon. 
        ts_lon =  coord[0]
        
        L = [] # Create a place to store the elevation.

        reshaped_coord= np.array([[ts_lon,ts_lat]]) #Input will be in lon, lat format. 


        hypot = cdist(L_locarray,reshaped_coord) #Get distance between input coords and coords in file.


        where_distance_small =  np.argmin(hypot) #Get the smallest distance index. 

        
        distance_dictionary[coord] = L_locarray[where_distance_small] #Access the lat/lon at the index.
        #We don't care if there's more than one closest coordinate, same dist in all directions. 

        convert_to_lst = L_locarray[where_distance_small] 
        convert_to_tuple = tuple(convert_to_lst) #Convert to tuple to look up value in dictionary. 
        
        min_distance_L[coord] = L_dictionary[convert_to_tuple] #Get value out of dictionary. 
        
    
    return min_distance_L

The following function is used to generate an array (in this case, it is basically a map) of 'b', which is a value that we need to calculate the overwinter Drought Code (DC) in the areas that need that. See this for more information: 
Lawson, B.D. & Armitage, O.B. (2008). Weather Guide for the Canadian Forest Fire Danger Rating System (pp. 1-84). Natural Resources Canada, Canadian Forest Service, Northern Forestry Centre. 

In [5]:
def get_b(latlon_dict,file_path_slope,idx_slope,file_path_drainage,idx_drainage,shapefile):
    '''Create a permanent lookup file for b for study area for future processing
    This is used in overwinter dc procedure 
    Parameters
        latlon_dict (dictionary, to be loaded from json file): dictionary of latitude and longitudes 
        for the hourly stations
        file_path_slope (str): path to the slope file, includes file name
        idx_slope (list): index of the slope variable in the header of the slope lookup file 
        file_path_drainage (str): path to the drainage file, includes file name
        idx_drainage (list): index of the drainage variable in the header of the drainage lookup file 
        shapefile (str): path to the shapefile of the study area (.shp format)
    Returns 
        The function will write a .json file to the hard drive, which can be loaded later. 
    '''

    lat = [] #Initialize empty lists to store data 
    lon = []
    for station_name in latlon_dict.keys(): #Loop through the list of stations 
        loc = latlon_dict[station_name]
        latitude = loc[0]
        longitude = loc[1]
        lat.append(float(latitude))
        lon.append(float(longitude))

    y = np.array(lat) #Convert to a numpy array for faster processing speed 
    x = np.array(lon)

    na_map = gpd.read_file(shapefile)
    bounds = na_map.bounds #Get the bounding box of the shapefile 
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']
    pixelHeight = 10000 #We want a 10 by 10 pixel, or as close as we can get 
    pixelWidth = 10000
            
    num_col = int((xmax - xmin) / pixelHeight) #Calculate the number of rows cols to fill the bounding box at that resolution 
    num_row = int((ymax - ymin) / pixelWidth)


    #We need to project to a projected system before making distance matrix
    source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume NAD83
    xProj, yProj = pyproj.Proj('esri:102001')(x,y) #Convert to Canada Albers Equal Area 

    yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']]) #Add the bounding box coords to the dataset so we can extrapolate the interpolation to cover whole area
    xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

    Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row) #Get the value for lat lon in each cell we just made 
    Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

    Xi,Yi = np.meshgrid(Xi,Yi) #Make a rectangular grid (because eventually we will map the values)
    Xi,Yi = Xi.flatten(), Yi.flatten() #Then we flatten the arrays for easier processing 
    #X and then Y for a reason 
    concat = np.array((Xi.flatten(), Yi.flatten())).T #Preparing the coordinates to send to the function that will get the elevation grid 
    send_to_list = concat.tolist()
    send_to_tuple = [tuple(x) for x in send_to_list] #The elevation function takes a tuple 

    #in cython dictionaries maintain insertion order 
    Xi1_grd=[]
    Yi1_grd=[]
    slope_grd = []
    drainage_grd = [] 
    slope_grd_dict = finding_data_frm_lookup(send_to_tuple,file_path_slope,idx_slope) #Get the elevations from the lookup file 
    drainage_grd_dict = finding_data_frm_lookup(send_to_tuple,file_path_drainage,idx_drainage)
    for keys in slope_grd_dict.keys(): #The keys are each lat lon pair 
        x= keys[0]
        y = keys[1]
        Xi1_grd.append(x)
        Yi1_grd.append(y)
        slope_grd.append(slope_grd_dict[keys])
        drainage_grd.append(drainage_grd_dict[keys])

    #combine the arrays
    slope_array = np.array(slope_grd)
    drainage_array = np.array(drainage_grd)
    
    #return the b array to be passed to the other function
    b_array = np.empty(slope_array.shape)
    b_array[drainage_array == 3] = 0.5
    b_array[drainage_array == 2] = 0.75
    b_array[drainage_array == 0] = 0.75
    b_array[drainage_array == 1] = 0.9
    b_array[slope_array > 0.5] = 0.5

    b_list = list(b_array)

    with open('b_list.json', 'w') as fp: #write to hard drive for faster processing later 
        json.dump(b_list, fp)

The following functions will need to be re-run repeatedly for each day to create the Drought Code, Duff Moisture Code, and Fine Fuel Moisture Code "stacks", which will be stored on the hard drive in .json format, so they can be re-accessed later without having to re-generate them. This means they will need to run quickly. I've implemented Cython here to try to optimize the code. 

In [6]:
%load_ext Cython

In [7]:
%%cython --annotate
from datetime import datetime #Needs to be re-imported inside the cell in Jupyter notebook 
import numpy as np
import feather
import pandas as pd
import os, sys

def get_wind_speed(str input_date,str file_path): 
    '''Create a dictionary for wind speed data on the input date 
    Parameters
        input_date (str): input date for the date of interest, in the format: YYYY-MM-DD HH:MM
        file_path (str): path to the feather files containing the hourly data from Environment & 
        Climate Change Canada 
    Returns 
        ws_dictionary (dict): a dictionary of wind speed values for all the active & non-null stations
        on the input date 
    '''
    cdef str station_name, file_name #Declare the C data types 
    cdef dict ws_dictionary = {}

    search_date = datetime.strptime(input_date, '%Y-%m-%d %H:%M') # Get the datetime object for input date

    for station_name in os.listdir(file_path):
        for file_name in os.listdir(file_path+station_name+'/'):
            if input_date[5:7] == file_name[29:31]: #This is a trick to speed up the code, only look at files which have the month/day in the name
                if input_date[0:4]== file_name[32:36]:
                    file = file_path+station_name+'/'+file_name

                    df = feather.read_dataframe(file)
                    try: 
                        if pd.notnull(df.loc[df['Date/Time'] == input_date, 'Wind Spd (km/h)'].item()):

                            #Put the value into the dictionary. 
                            ws_dictionary[station_name] = df.loc[df['Date/Time'] == input_date, 'Wind Spd (km/h)'].item()

                        else: 
                            pass
                    except ValueError:
                        pass 

    return ws_dictionary

You can run the cell below to see an example of the output and how long the function takes. 

In [8]:
%%cython
from datetime import datetime #Needs to be re-imported inside the cell in Jupyter notebook 
import numpy as np
import feather
import pandas as pd
import os, sys

def get_noon_temp(str input_date,str file_path):
    '''Create a dictionary for noon temp data on the input date 
    Parameters
        input_date (str): input date for the date of interest, in the format: YYYY-MM-DD HH:MM
        file_path (str): path to the feather files containing the hourly data from Environment & 
        Climate Change Canada 
    Returns 
        temp_dictionary (dict): a dictionary of temperature values for all the active & non-null stations
        on the input date 
    '''
    cdef str station_name, file_name
    cdef dict temp_dictionary = {}
    
    search_date = datetime.strptime(input_date, '%Y-%m-%d %H:%M')


    for station_name in os.listdir(file_path):
        for file_name in os.listdir(file_path+station_name+'/'):
            if input_date[5:7] == file_name[29:31]:
                if input_date[0:4]== file_name[32:36]:
                    file = file_path+station_name+'/'+file_name

                    df = feather.read_dataframe(file)

                    try: 
                        if pd.notnull(df.loc[df['Date/Time'] == input_date, 'Temp (Â°C)'].item()):


                            temp_dictionary[station_name] = df.loc[df['Date/Time'] == input_date, 'Temp (Â°C)'].item()

                        else: 
                            pass
                    except ValueError:
                        pass 

    return temp_dictionary

In [9]:
%%cython
from datetime import datetime #Needs to be re-imported inside the cell in Jupyter notebook 
import numpy as np
import feather
import pandas as pd
import os, sys

def get_relative_humidity(str input_date,str file_path):
    '''Create a dictionary for rh% data on the input date 
    Parameters
        input_date (str): input date for the date of interest, in the format: YYYY-MM-DD HH:MM
        file_path (str): path to the feather files containing the hourly data from Environment & 
        Climate Change Canada 
    Returns 
        temp_dictionary (dict): a dictionary of relative humidity values for all the active & 
        non-null stations on the input date 
    '''
    cdef str station_name, file_name
    cdef dict RH_dictionary = {}

    search_date = datetime.strptime(input_date, '%Y-%m-%d %H:%M')


    for station_name in os.listdir(file_path):
        for file_name in os.listdir(file_path+station_name+'/'):
            if input_date[5:7] == file_name[29:31]:
                if input_date[0:4]== file_name[32:36]:
                    file = file_path+station_name+'/'+file_name

                    df = feather.read_dataframe(file)
                    try: 
                        if pd.notnull(df.loc[df['Date/Time'] == input_date, 'Rel Hum (%)'].item()):


                            RH_dictionary[station_name] = df.loc[df['Date/Time'] == input_date, 'Rel Hum (%)'].item()

                        else: 
                            pass
                    except ValueError:
                        pass 

    return RH_dictionary


In [10]:
def get_pcp_dictionary_by_year(file_path):
    '''Create a lookup file for the year_month that each daily station has data for faster 
    processing later --> this is an input to get_pcp 
    Parameters
        file_path (str): file path to the daily csv files provided by Environment & Climate Change
        Canada, including the name of the file 
    Returns 
        The function writes a .json file to the hard drive that can be read back into the code before
        running get_pcp 
    '''
    
    date_dictionary = {}
    for station_name in os.listdir(file_path):
        
        yearList = []
        count = 0
        with open(file_path+station_name, encoding='latin1') as year_information:
            for row in year_information:
                information = row.rstrip('\n').split(',')
                information_stripped = [i.replace('"','') for i in information]
                if count==0:
                    
                    header= information_stripped

                    keyword = 'month' #There is also the flag which is why we include the (
                    idx_list = [i for i, x in enumerate(header) if keyword in x.lower()]
                    if len(idx_list) >1:
                        print('The program is confused because there is more than one field name that could \
                        contain the month. Please check on this.') #there could be no index if the file is empty, which sometimes happens 
                        sys.exit()

                    keyword2 = 'year'
                    idx_list2 = [i for i, x in enumerate(header) if keyword2 in x.lower()]
                    if len(idx_list2) > 1: # There should only be one field 
                        print('The program is confused because there is more than one field name that could \
                        contain the year. Please check on this.')
                        sys.exit()
                if count > 0:
                    if int(information_stripped[idx_list[0]]) >= 10: 
                        year_month = str(int(information_stripped[idx_list2[0]]))+'-'+str(int(information_stripped[idx_list[0]]))
                    else:
                        year_month = str(int(information_stripped[idx_list2[0]]))+'-0'+str(int(information_stripped[idx_list[0]]))
                    if year_month not in yearList: 
                        yearList.append(year_month)
                count+=1
        date_dictionary[station_name[:-4]] =yearList 

    with open('daily_lookup_file_TEMP.json', 'w') as fp:
        json.dump(date_dictionary, fp)

In [11]:
def get_daily_lat_lon(file_path):
    '''Get the latitude and longitude of the daily stations and store in a json file in the directory
    Parameters
        file_path (str): file path to the daily csv files provided by Environment & Climate Change
        Canada, including the name of the file 
    Returns 
        The function writes a .json file to the hard drive that can be read back into the code before
        running get_pcp 
    '''
    latlon_dictionary = {} 
    #for station_name in lat_lon_list: #This is the list of available stations on that day
    for station_name in os.listdir(file_path):
        latlon_list = []
        with open(file_path+station_name, encoding='latin1') as latlon_information:
            
            count=0
            for row in latlon_information:
                
                information = row.rstrip('\n').split(',')
                information_stripped = [i.replace('"','') for i in information]
                
                if count==0:
                    header= information_stripped #We will look for latitude and longitude keywords in the header and find the index
                    keyword = 'lon'
                    idx_list = [i for i, x in enumerate(header) if keyword in x.lower()]
                    keyword2 = 'lat'
                    idx_list2 = [i for i, x in enumerate(header) if keyword2 in x.lower()]
                    if len(idx_list) > 1: # There should only be one field 
                        print('The program is confused because there is more than one field name that could \
                        contain the longitude. Please check on this.')
                        sys.exit()
                    if len(idx_list2) > 1: # There should only be one field 
                        print('The program is confused because there is more than one field name that could \
                        contain the latitude. Please check on this.')
                        sys.exit()
                        
                if count == 1:
                    if float(information_stripped[idx_list2[0]]) != 0 or float(information_stripped[idx_list[0]]) != 0: #sometimes lat and lon is 0, need to exclude

                        latlon_list.append((information_stripped[idx_list2[0]],information_stripped[idx_list[0]]))
                        break
                    else:
                        pass 
                    
                count+=1
                
        if len(set(latlon_list)) > 1:
            print('For %s there is more than one location in the list! You can only have one record per station so please check the data.'%(station_name))
        elif len(set(latlon_list)) == 0:
            print('A valid lat lon for that station was not found in the file.') 
        else:
            try: 
                latlon_dictionary[station_name[:-4]]=latlon_list[0]

            except:
                print('There is a problem with the files for %s and the location has not been recorded. Please check.'%(station_name))
    with open('daily_lat_lon_TEMP.json', 'w') as fp:
        json.dump(latlon_dictionary, fp)

In [12]:
%%cython
from datetime import datetime #Needs to be re-imported inside the cell in Jupyter notebook 
import numpy as np
import feather
import pandas as pd
import os, sys

def get_pcp(str input_date,str file_path,dict date_dictionary):
    '''Get a dictionary of the precipitation data from the feather files of the daily stations 
    Parameters
        input_date (str): input date for the day of interest, in the format YYYY:MM:DD 
        file_path (str): file path to the daily feather files 
        date_dictionary (dict, loaded from .json): lookup file that has what day/month pairs each 
        station is active on 
    Returns 
        rain_dictionary (dict): dictionary containing rain amount for each station 
    '''

    cdef str station_name, yearMonth
    cdef dict rain_dictionary = {}

    yearMonth = input_date[0:7]

    for station_name in os.listdir(file_path):
        yearsMonths = date_dictionary[station_name[:-8]] #-8 bc we are now working with feather files 

        if yearMonth in yearsMonths:

            file = file_path+station_name

            df = feather.read_dataframe(file)
            try: 
                if pd.notnull(df.loc[df['date'] == input_date, 'total_precip'].item()):


                    rain_dictionary[station_name[:-8]] = df.loc[df['date'] == input_date, 'total_precip'].item()

                else: 
                    pass
            except ValueError:
                pass #is this for trace precip?

    return rain_dictionary

In [14]:
def get_lat_lon(file_path):
    '''Get the latitude and longitude of the hourly stations and write to hard drive as a json file 
    Parameters
        file_path (str): file path to the hourly csv files provided by Environment & Climate Change
        Canada, including the name of the file 
    Returns 
        The function writes a .json file to the hard drive that can be read back into the code before
        running get_pcp 
    '''
    latlon_dictionary = {} 

    for station_name in os.listdir(file_path):
        latlon_list = []
        files = os.listdir(file_path+station_name+'/')[0]
        with open(file_path+station_name+'/'+files, encoding='latin1') as latlon_information:
            print(station_name)
            count=0
            for row in latlon_information:
                
                information = row.rstrip('\n').split(',')
                information_stripped = [i.replace('"','') for i in information]
                
                if count==0:
                    header= information_stripped #We will look for latitude and longitude keywords in the header and find the index
                    keyword = 'longitude'
                    idx_list = [i for i, x in enumerate(header) if keyword in x.lower()]
                    keyword2 = 'latitude'
                    idx_list2 = [i for i, x in enumerate(header) if keyword2 in x.lower()]
                    if len(idx_list) > 1: # There should only be one field 
                        print('The program is confused because there is more than one field name that could \
                        contain the longitude. Please check on this.')
                        sys.exit()
                    if len(idx_list2) > 1: # There should only be one field 
                        print('The program is confused because there is more than one field name that could \
                        contain the latitude. Please check on this.')
                        sys.exit()
                        
                if count == 1:
                    latlon_list.append((information_stripped[idx_list2[0]],information_stripped[idx_list[0]]))

                    break 
                    
                count+=1
                
        if len(set(latlon_list)) > 1:
            print('For %s there is more than one location in the list! You can only have one record per station so please check the data.'%(station_name))
        else:
            try: 
                latlon_dictionary[station_name]=latlon_list[0]
            except:
                print('There is a problem with the files for %s and the location has not been recorded. Please check.'%(station_name))

    with open('hourly_lat_lon_TEMP.json', 'w') as fp:
        json.dump(latlon_dictionary, fp)

The next group of functions (see below) are used for calculating the fire season start up and end dates for each individual station with an unbroken temperature record. 

In [15]:
#Import functions needed for the next group of functions 
import geopandas as gpd
import numpy as np
import pyproj
import matplotlib.pyplot as plt
from itertools import groupby
from datetime import datetime, timedelta, date
import pandas as pd
import math
import os, sys
import gc
import feather 

In [16]:
#Using the feather fires, which are copied faster to another computer, but the code is slower than
#if we use the csv files 
def start_date_calendar(file_path_hourly,year):
    '''Returns a dictionary of where each station meets the start up criteria, plus a reference dictionary for the lat lon of the stations
    Parameters
        file_path (str): path to the feather files containing the hourly data from Environment & 
        Climate Change Canada 
        year (str): year we want to find the fire season start up date for 
    Returns 
        date_dict (dict): dictionary containing the start up date for each station (days since Mar 1)
        latlon_dictionary (dict): the latitude and longitude of those stations 
    '''

    #Input: path to hourly data, string of the year, i.e. '1998' 
    maxTempList_dict = {} #Locations where we will store the data
    maxTemp_dictionary = {}
    date_dict = {}
    latlon_dictionary = {}

    for station_name in os.listdir(file_path_hourly): #The dictionary will be keyed by the hourly (temperature) station names, which means all the names must be unique
        Temp_subdict = {} #We will need an empty dictionary to store the data due to data ordering issues 
        temp_list = [] #Initialize an empty list to temporarily store data we will later send to a permanent dictionary 
        for csv in os.listdir(file_path_hourly+station_name+'/'): #Loop through the csv in the station folder
            if year in csv: #Only open if it is the csv for the year of interest (this is contained in the csv name)
                file = file_path_hourly+station_name+'/'+csv  #Open the file - for CAN data we use latin 1 due to à, é etc.
                df = feather.read_dataframe(file)

                count = 0 

                for index, row in df.iterrows():
                    if count == 0:
                        
                        latlon_dictionary[station_name] = (row['Latitude (y)'], row['ï»¿"Longitude (x)"']) #unicode characters at beginning, not sure why 
                    if str(row['Year']) == year:

                        if str(row['Month']) == '3' or str(row['Month']) == '4' or str(row['Month']) == '5' or \
                           str(row['Month']) == '6' or str(row['Month']) == '7':
                            
                            if str(row['Time']) == '13:00':

                                if pd.notnull(row['Temp (Â°C)']):

                                    Temp_subdict[str(row['Date/Time'])] = float(row['Temp (Â°C)'])
                                    temp_list.append(float(row['Temp (Â°C)'])) #Get the 13h00 temperature, send to temp list
                                else:
                                    Temp_subdict[str(row['Date/Time'])] = 'NA'
                                    temp_list.append('NA')
                    count +=1 

        maxTemp_dictionary[station_name] = Temp_subdict
        maxTempList_dict[station_name] = temp_list #Store the information for each station in the permanent dictionary 

        vals = maxTempList_dict[station_name]

        if 'NA' not in vals and len(vals) == 153: #only consider the stations with unbroken records, num_days between March-July is 153

            varray = np.array(vals)
            where_g12 = np.array(varray >= 12) #Where is the temperature >=12? 


            groups = [list(j) for i, j in groupby(where_g12)] #Put the booleans in groups, ex. [True, True], [False, False, False] 

            length = [x for x in groups if len(x) >= 3 and x[0] == True] #Obtain a list of where the groups are three or longer which corresponds to at least 3 days >= 12


            index = groups.index(length[0]) #Get the index of the group
            group_len = [len(x) for x in groups] #Get length of each group
            length_sofar = 0 #We need to get the number of days up to where the criteria is met 
            for i in range(0,index): #loop through each group until you get to the index and add the length of that group 
                length_sofar += group_len[i]

            Sdate = list(sorted(maxTemp_dictionary[station_name].keys()))[length_sofar+2] #Go two days ahead for the third day 

            d0 = date(int(year), 3, 1) #March 1, Year 
            d1 = date(int(Sdate[0:4]), int(Sdate[5:7]), int(Sdate[8:10])) #Convert to days since march 1 so we can interpolate
            delta = d1 - d0
            day = int(delta.days) #Convert to integer 
            date_dict[station_name] = day #Store the integer in the dictionary 


            #print('The start date for %s for %s is %s'%(station_name,year,Sdate))

    #Return the dates for each station

    return date_dict, latlon_dictionary 

In [17]:
%%cython

import os, sys
import numpy as np 
from itertools import groupby
from datetime import datetime, timedelta, date

#Same function as above but will read directly from the csv files, which is faster 
def start_date_calendar_csv(file_path_hourly,year):
    '''Returns a dictionary of where each station meets the start up criteria, plus a reference dictionary for the lat lon of the stations
    Parameters
        file_path (str): path to the csv files containing the hourly data from Environment & 
        Climate Change Canada 
        year (str): year we want to find the fire season start up date for 
    Returns 
        date_dict (dict): dictionary containing the start up date for each station (days since Mar 1)
        latlon_dictionary (dict): the latitude and longitude of those stations 
    '''

    #Input: path to hourly data, string of the year, i.e. '1998' 
    maxTempList_dict = {} #Locations where we will store the data
    maxTemp_dictionary = {}
    date_dict = {}
    latlon_dictionary = {}

    for station_name in os.listdir(file_path_hourly): #The dictionary will be keyed by the hourly (temperature) station names, which means all the names must be unique
        Temp_subdict = {} #We will need an empty dictionary to store the data due to data ordering issues 
        temp_list = [] #Initialize an empty list to temporarily store data we will later send to a permanent dictionary 
        count=0
        for csv in os.listdir(file_path_hourly+station_name+'/'): #Loop through the csv in the station folder 
            if year in csv: #Only open if it is the csv for the year of interest (this is contained in the csv name)

                with open(file_path_hourly+station_name+'/'+csv, encoding='latin1') as year_information: #Open the file - for CAN data we use latin 1 due to à, é etc. 
                    for row in year_information: #Look at each row 
                        information = row.rstrip('\n').split(',') #Split each row into a list so we can loop through 
                        information_stripped = [i.replace('"','') for i in information] #Get rid of extra quotes in the header
                        if count==0: #This is getting the first row 
                            
                            header= information_stripped


                            keyword = 'temp (' #Look for this keyword in the header 
                            filter_out_keyword = 'dew' #We don't want dewpoint temperature, we want to skip over it 
                            idx_list1 = [i for i, x in enumerate(header) if keyword in x.lower() and filter_out_keyword not in x.lower()] #Get the index of the temperature column

                            if len(idx_list1) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the temp data. Please check on this.')
                                sys.exit()
                            keyword2 = 'date/time' #Getting the index of the datetime object so we can later make sure we are using 13h00 value 
                            idx_list2 = [i for i, x in enumerate(header) if keyword2 in x.lower()]

                            if len(idx_list2) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the date. Please check on this.')
                                sys.exit()

                            keyword3 = 'latitude' #Here we use the same methods to get the latitude and longitude 
                            idx_list3 = [i for i, x in enumerate(header) if keyword3 in x.lower()]
                            if len(idx_list3) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the latitude. Please check on this.')
                                sys.exit()
                            keyword4 = 'longitude'
                            idx_list4 = [i for i, x in enumerate(header) if keyword4 in x.lower()]
                            if len(idx_list4) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the latitude. Please check on this.')
                                sys.exit()
                                
                        if count > 0: #Now we are looking at the rest of the file, after the header 

                            if count == 1: #Lat/lon will be all the same so only record it once
                                try: #If the file is corrupted (it usually looks like a bunch of random characters) we will get an error, so we need a try/except loop
                                    lat =float(information_stripped[idx_list3[0]])
                                    lon =float(information_stripped[idx_list4[0]])
                                    latlon_dictionary[station_name] = tuple((lat,lon)) #Get the lat lon and send the tuple to the dictionary 
                                except:
                                    print('Something is wrong with the lat/lon header names for %s!'%(station_name))
                                    break 
 
          
                                try:
                                    if information_stripped[idx_list2[0]][0:4] == year: #Make sure we have the right year 
                                        if information_stripped[idx_list2[0]][5:7] == '03' or information_stripped[idx_list2[0]][5:7] == '04' or \
                                           information_stripped[idx_list2[0]][5:7] == '05' or information_stripped[idx_list2[0]][5:7] == '06' or \
                                           information_stripped[idx_list2[0]][5:7] == '07': #Make sure we are only considering months since March in case of heat wave in another month
                                            if information_stripped[idx_list2[0]][11:13] == '13': #We are only interested in checking the 13h00 temperature
                                                Temp_subdict[information_stripped[idx_list2[0]]] = float(information_stripped[idx_list1[0]])
                                                temp_list.append(float(information_stripped[idx_list1[0]])) #Get the 13h00 temperature, send to temp list
                                            


                                except: #In the case of a nodata value
                                    Temp_subdict[information_stripped[idx_list2[0]]] = 'NA'
                                    temp_list.append('NA')
                                    

                            else: #Proceed down the rows 
                                try:

                                    if information_stripped[idx_list2[0]][0:4] == year: 
                                        if information_stripped[idx_list2[0]][5:7] == '03' or information_stripped[idx_list2[0]][5:7] == '04' or information_stripped[idx_list2[0]][5:7] == '05'\
                                           or information_stripped[idx_list2[0]][5:7] == '06' or information_stripped[idx_list2[0]][5:7] == '07':
                                            if information_stripped[idx_list2[0]][11:13] == '13':
                                                Temp_subdict[information_stripped[idx_list2[0]]] = float(information_stripped[idx_list1[0]])
                                                temp_list.append(float(information_stripped[idx_list1[0]]))



                                except:
                                    Temp_subdict[information_stripped[idx_list2[0]]] = 'NA'
                                    temp_list.append('NA')

                        count+=1   

        maxTemp_dictionary[station_name] = Temp_subdict
        maxTempList_dict[station_name] = temp_list #Store the information for each station in the permanent dictionary 

        vals = maxTempList_dict[station_name]

        if 'NA' not in vals and len(vals) == 153: #only consider the stations with unbroken records, num_days between March-July is 153

            varray = np.array(vals)
            where_g12 = np.array(varray >= 12) #Where is the temperature >=12? 


            groups = [list(j) for i, j in groupby(where_g12)] #Put the booleans in groups, ex. [True, True], [False, False, False] 

            length = [x for x in groups if len(x) >= 3 and x[0] == True] #Obtain a list of where the groups are three or longer which corresponds to at least 3 days >= 12


            index = groups.index(length[0]) #Get the index of the group
            group_len = [len(x) for x in groups] #Get length of each group
            length_sofar = 0 #We need to get the number of days up to where the criteria is met 
            for i in range(0,index): #loop through each group until you get to the index and add the length of that group 
                length_sofar += group_len[i]

            Sdate = list(sorted(maxTemp_dictionary[station_name].keys()))[length_sofar+2] #Go two days ahead for the third day 

            d0 = date(int(year), 3, 1) #March 1, Year 
            d1 = date(int(Sdate[0:4]), int(Sdate[5:7]), int(Sdate[8:10])) #Convert to days since march 1 so we can interpolate
            delta = d1 - d0
            day = int(delta.days) #Convert to integer 
            date_dict[station_name] = day #Store the integer in the dictionary 


            #print('The start date for %s for %s is %s'%(station_name,year,Sdate))

    #Return the dates for each station 
    return date_dict, latlon_dictionary 

For more information about the end date calculation, please see: 

Wotton, B. M., & Flannigan, M. D. (1993). Length of the fire season in a changing climate. Forestry Chronicle, 69(2), 187–192. https://doi.org/10.5558/tfc69187-2

In [18]:
def end_date_calendar(file_path_hourly,year):
    '''Returns a dictionary of where each station meets the start up criteria, 
    plus a reference dictionary for the lat lon of the stations
    Parameters
        file_path (str): path to the feather files containing the hourly data from Environment & 
        Climate Change Canada 
        year (str): year we want to find the fire season end date for 
    Returns 
        date_dict (dict): dictionary containing the end date for each station (days since Oct 1)
        latlon_dictionary (dict): the latitude and longitude of those stations 
    '''
    #Input: path to hourly data, string of the year, i.e. '1998'
    maxTempList_dict = {} #Locations where we will store the data
    maxTemp_dictionary = {}
    date_dict = {}
    latlon_dictionary = {}
    for station_name in os.listdir(file_path_hourly): #The dictionary will be keyed by the hourly (temperature) station names, which means all the names must be unique
        Temp_subdict = {} #We will need an empty dictionary to store the data due to data ordering issues 
        temp_list = [] #Initialize an empty list to temporarily store data we will later send to a permanent dictionary 
        for csv in os.listdir(file_path_hourly+station_name+'/'): #Loop through the csv in the station folder
            if year in csv: #Only open if it is the csv for the year of interest (this is contained in the csv name)
                file = file_path_hourly+station_name+'/'+csv  #Open the file - for CAN data we use latin 1 due to à, é etc.
                df = feather.read_dataframe(file)

                count = 0 

                for index, row in df.iterrows():
                    if count == 0:
                        
                        latlon_dictionary[station_name] = (row['Latitude (y)'], row['ï»¿"Longitude (x)"']) #unicode characters at beginning, not sure why 
                    if str(row['Year']) == year:

                        if str(row['Month']) == '10' or str(row['Month']) == '11' or str(row['Month']) == '12':

                            if str(row['Time']) == '13:00':

                                if pd.notnull(row['Temp (Â°C)']):
                                    Temp_subdict[row['Date/Time']] = float(row['Temp (Â°C)'])
                                    temp_list.append(float(row['Temp (Â°C)'])) #Get the 13h00 temperature, send to temp list
                                else:
                                    Temp_subdict[row['Date/Time']] = 'NA'
                                    temp_list.append('NA')
                    count +=1 

                    
        maxTemp_dictionary[station_name] = Temp_subdict
        maxTempList_dict[station_name] = temp_list #Store the information for each station in the permanent dictionary
        vals = maxTempList_dict[station_name]

        
        if 'NA' not in vals and len(vals) == 92: #only consider the stations with unbroken records, num_days between Oct1-Dec31 = 92

            varray = np.array(vals)
            where_g12 = np.array(varray < 5) #Where is the temperature < 5? 


            groups = [list(j) for i, j in groupby(where_g12)] #Put the booleans in groups, ex. [True, True], [False, False, False] 

            length = [x for x in groups if len(x) >= 3 and x[0] == True] #Obtain a list of where the groups are three or longer which corresponds to at least 3 days < 5

            
            index = groups.index(length[0]) #Get the index of the group
            group_len = [len(x) for x in groups] #Get length of each group
            length_sofar = 0 #We need to get the number of days up to where the criteria is met 
            for i in range(0,index): #loop through each group until you get to the index and add the length of that group 
                length_sofar += group_len[i]

            Sdate = list(sorted(maxTemp_dictionary[station_name].keys()))[length_sofar+2] #Go two days ahead for the third day 

            d0 = date(int(year), 10, 1) #Oct 1, Year 
            d1 = date(int(Sdate[0:4]), int(Sdate[5:7]), int(Sdate[8:10])) #Convert to days since Oct 1 so we can interpolate
            delta = d1 - d0
            day = int(delta.days) #Convert to integer 
            date_dict[station_name] = day #Store the integer in the dictionary 


            #print('The end date for %s for %s is %s'%(station_name,year,Sdate))

    #Return the dates for each station
    return date_dict, latlon_dictionary

In [19]:
%%cython

import os, sys
import numpy as np 
from itertools import groupby
from datetime import datetime, timedelta, date

def end_date_calendar_csv(str file_path_hourly,str year):
    '''Returns a dictionary of where each station meets the end criteria see 
    Wotton & Flannigan 1993, plus a reference dictionary for the lat lon of the stations
    Parameters
        file_path (str): path to the csv files containing the hourly data from Environment & 
        Climate Change Canada 
        year (str): year we want to find the fire season end date for 
    Returns 
        date_dict (dict): dictionary containing the end date for each station (days since Oct 1)
        latlon_dictionary (dict): the latitude and longitude of those stations 
    '''
    #Input: path to hourly data, string of the year, i.e. '1998' 
    maxTempList_dict = {} #Locations where we will store the data
    maxTemp_dictionary = {}
    date_dict = {}
    latlon_dictionary = {}

    for station_name in os.listdir(file_path_hourly): #The dictionary will be keyed by the hourly (temperature) station names, which means all the names must be unique
        Temp_subdict = {} #We will need an empty dictionary to store the data due to data ordering issues 
        temp_list = [] #Initialize an empty list to temporarily store data we will later send to a permanent dictionary 
        count=0
        for csv in os.listdir(file_path_hourly+station_name+'/'): #Loop through the csv in the station folder 
            if year in csv: #Only open if it is the csv for the year of interest (this is contained in the csv name)

                with open(file_path_hourly+station_name+'/'+csv, encoding='latin1') as year_information: #Open the file - for CAN data we use latin 1 due to à, é etc. 
                    for row in year_information: #Look at each row 
                        information = row.rstrip('\n').split(',') #Split each row into a list so we can loop through 
                        information_stripped = [i.replace('"','') for i in information] #Get rid of extra quotes in the header
                        if count==0: #This is getting the first row 
                            
                            header= information_stripped


                            keyword = 'temp (' #Look for this keyword in the header 
                            filter_out_keyword = 'dew' #We don't want dewpoint temperature, we want to skip over it 
                            idx_list1 = [i for i, x in enumerate(header) if keyword in x.lower() and filter_out_keyword not in x.lower()] #Get the index of the temperature column

                            if len(idx_list1) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the temp data. Please check on this.')
                                sys.exit()
                            keyword2 = 'date/time' #Getting the index of the datetime object so we can later make sure we are using 13h00 value 
                            idx_list2 = [i for i, x in enumerate(header) if keyword2 in x.lower()]

                            if len(idx_list2) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the date. Please check on this.')
                                sys.exit()

                            keyword3 = 'latitude' #Here we use the same methods to get the latitude and longitude 
                            idx_list3 = [i for i, x in enumerate(header) if keyword3 in x.lower()]
                            if len(idx_list3) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the latitude. Please check on this.')
                                sys.exit()
                            keyword4 = 'longitude'
                            idx_list4 = [i for i, x in enumerate(header) if keyword4 in x.lower()]
                            if len(idx_list4) > 1: # There should only be one field 
                                print('The program is confused because there is more than one field name that could \
                                contain the latitude. Please check on this.')
                                sys.exit()
                                
                        if count > 0: #Now we are looking at the rest of the file, after the header 

                            if count == 1: #Lat/lon will be all the same so only record it once
                                try: #If the file is corrupted (it usually looks like a bunch of random characters) we will get an error, so we need a try/except loop
                                    lat =float(information_stripped[idx_list3[0]])
                                    lon =float(information_stripped[idx_list4[0]])
                                    latlon_dictionary[station_name] = tuple((lat,lon)) #Get the lat lon and send the tuple to the dictionary 
                                except:
                                    print('Something is wrong with the lat/lon header names for %s!'%(station_name))
                                    break 
 
          
                                try:
                                    if information_stripped[idx_list2[0]][0:4] == year: #Make sure we have the right year 
                                        if information_stripped[idx_list2[0]][5:7] == '10' or information_stripped[idx_list2[0]][5:7] == '11' or information_stripped[idx_list2[0]][5:7] == '12': #Make sure we are only considering months after October 
                                            if information_stripped[idx_list2[0]][11:13] == '13': #We are only interested in checking the 13h00 temperature
                                                Temp_subdict[information_stripped[idx_list2[0]]] = float(information_stripped[idx_list1[0]])
                                                temp_list.append(float(information_stripped[idx_list1[0]])) #Get the 13h00 temperature, send to temp list
                                            


                                except: #In the case of a nodata value
                                    Temp_subdict[information_stripped[idx_list2[0]]] = 'NA'
                                    temp_list.append('NA')
                                    

                            else: #Proceed down the rows 
                                try:

                                    if information_stripped[idx_list2[0]][0:4] == year: 
                                        if information_stripped[idx_list2[0]][5:7] == '10' or information_stripped[idx_list2[0]][5:7] == '11' or information_stripped[idx_list2[0]][5:7] == '12':
                                            if information_stripped[idx_list2[0]][11:13] == '13':
                                                Temp_subdict[information_stripped[idx_list2[0]]] = float(information_stripped[idx_list1[0]])
                                                temp_list.append(float(information_stripped[idx_list1[0]]))



                                except:
                                    Temp_subdict[information_stripped[idx_list2[0]]] = 'NA'
                                    temp_list.append('NA')

                        count+=1   

        maxTemp_dictionary[station_name] = Temp_subdict
        maxTempList_dict[station_name] = temp_list #Store the information for each station in the permanent dictionary 

        vals = maxTempList_dict[station_name]

        if 'NA' not in vals and len(vals) == 92: #only consider the stations with unbroken records, num_days between Oct1-Dec31 = 92

            varray = np.array(vals)
            where_g12 = np.array(varray < 5) #Where is the temperature < 5? 


            groups = [list(j) for i, j in groupby(where_g12)] #Put the booleans in groups, ex. [True, True], [False, False, False] 

            length = [x for x in groups if len(x) >= 3 and x[0] == True] #Obtain a list of where the groups are three or longer which corresponds to at least 3 days < 5

            
            index = groups.index(length[0]) #Get the index of the group
            group_len = [len(x) for x in groups] #Get length of each group
            length_sofar = 0 #We need to get the number of days up to where the criteria is met 
            for i in range(0,index): #loop through each group until you get to the index and add the length of that group 
                length_sofar += group_len[i]

            Sdate = list(sorted(maxTemp_dictionary[station_name].keys()))[length_sofar+2] #Go two days ahead for the third day 

            d0 = date(int(year), 10, 1) #Oct 1, Year 
            d1 = date(int(Sdate[0:4]), int(Sdate[5:7]), int(Sdate[8:10])) #Convert to days since Oct 1 so we can interpolate
            delta = d1 - d0
            day = int(delta.days) #Convert to integer 
            date_dict[station_name] = day #Store the integer in the dictionary 


            #print('The end date for %s for %s is %s'%(station_name,year,Sdate))

    #Return the dates for each station 
    return date_dict, latlon_dictionary

The following code is for spatial interpolation and leave-one-out cross-validation. 

In [20]:
import geopandas as gpd
import numpy as np
import pyproj
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore") #Runtime warning suppress, this suppresses the /0 warning

In [21]:
%%cython
import numpy as np
import os, sys
import geopandas as gpd
import pyproj
import matplotlib.pyplot as plt

def IDW(latlon_dict,Cvar_dict,str input_date,str var_name,str shapefile,bint show,int d): #See IDEW for comments
     '''Inverse distance weighting interpolation
     Parameters
         latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
         .json file
         Cvar_dict (dict): dictionary of weather variable values for each station 
         input_date (str): the date you want to interpolate for 
         shapefile (str): path to the study area shapefile 
         show (bool): whether you want to plot a map 
         d (int): the weighting function for IDW interpolation 
     Returns
         idw_grid (np_array): the array of values for the interpolated surface
         maxmin: the bounds of the array surface, for use in other functions 
     '''
     lat = []
     lon = []
     Cvar = []
     for station_name in Cvar_dict.keys():

        if station_name in latlon_dict.keys():

            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            cvar_val = Cvar_dict[station_name]
            lat.append(float(latitude))
            lon.append(float(longitude))
            Cvar.append(cvar_val)
     y = np.array(lat)
     x = np.array(lon)
     z = np.array(Cvar) 

     na_map = gpd.read_file(shapefile)
     bounds = na_map.bounds
     xmax = bounds['maxx']
     xmin= bounds['minx']
     ymax = bounds['maxy']
     ymin = bounds['miny']
     cdef int pixelHeight = 10000 
     cdef int pixelWidth = 10000
            
     cdef int num_col = int((xmax - xmin) / pixelHeight)
     cdef int num_row = int((ymax - ymin) / pixelWidth)


     #We need to project to a projected system before making distance matrix
     source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') 
     xProj, yProj = pyproj.Proj('esri:102001')(x,y)

     yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
     xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

     Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
     Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

     Xi,Yi = np.meshgrid(Xi,Yi)
     Xi,Yi = Xi.flatten(), Yi.flatten()
     maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)]

     vals = np.vstack((xProj,yProj)).T

     interpol = np.vstack((Xi,Yi)).T
     dist_not = np.subtract.outer(vals[:,0], interpol[:,0]) 
     dist_one = np.subtract.outer(vals[:,1], interpol[:,1]) 
     distance_matrix = np.hypot(dist_not,dist_one) 

     weights = 1/(distance_matrix **d)
     weights[np.where(np.isinf(weights))] = 1/(1.0E-50) 
     weights /= weights.sum(axis = 0) 

     Zi = np.dot(weights.T, z)
     idw_grid = Zi.reshape(num_row,num_col)

     if show:
        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(idw_grid,extent=(xProj_extent.min()-1,xProj_extent.max()+1,yProj_extent.max()-1,yProj_extent.min()+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.scatter(xProj,yProj,c=z,edgecolors='k')

        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label(var_name) 
        
        title = 'IDW Interpolation for %s on %s'%(var_name,input_date)
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude') 

        plt.show()


     return idw_grid, maxmin

In [24]:
def cross_validate_IDW(latlon_dict,Cvar_dict,shapefile,d):
     '''Leave-one-out cross-validation procedure for IDW 
     Parameters
         latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
         .json file
         Cvar_dict (dict): dictionary of weather variable values for each station 
         shapefile (str): path to the study area shapefile 
         d (int): the weighting function for IDW interpolation 
     Returns 
         absolute_error_dictionary (dict): a dictionary of the absolute error at each station when it
         was left out 
     '''
     x_origin_list = []
     y_origin_list = [] 

     absolute_error_dictionary = {} #for plotting
     station_name_list = []
     projected_lat_lon = {}

     for station_name in Cvar_dict.keys():
          
          if station_name in latlon_dict.keys():
               
             station_name_list.append(station_name)

             loc = latlon_dict[station_name]
             latitude = loc[0]
             longitude = loc[1]
             Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
             Plat = float(Plat)
             Plon = float(Plon)
             projected_lat_lon[station_name] = [Plat,Plon]



     for station_name_hold_back in station_name_list:

        lat = []
        lon = []
        Cvar = []
        for station_name in sorted(Cvar_dict.keys()):
             if station_name in latlon_dict.keys():
                 if station_name != station_name_hold_back:
                     loc = latlon_dict[station_name]
                     latitude = loc[0]
                     longitude = loc[1]
                     cvar_val = Cvar_dict[station_name]
                     lat.append(float(latitude))
                     lon.append(float(longitude))
                     Cvar.append(cvar_val)
                 else:

                     pass
                
        y = np.array(lat)
        x = np.array(lon)
        z = np.array(Cvar) 
        
        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds
        xmax = bounds['maxx']
        xmin= bounds['minx']
        ymax = bounds['maxy']
        ymin = bounds['miny']
        pixelHeight = 10000 
        pixelWidth = 10000
                
        num_col = int((xmax - xmin) / pixelHeight)
        num_row = int((ymax - ymin) / pixelWidth)


        #We need to project to a projected system before making distance matrix
        source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
        xProj, yProj = pyproj.Proj('esri:102001')(x,y)

        yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
        xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

        Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
        Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

        Xi,Yi = np.meshgrid(Xi,Yi)
        Xi,Yi = Xi.flatten(), Yi.flatten()
        maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)]

        vals = np.vstack((xProj,yProj)).T
        
        interpol = np.vstack((Xi,Yi)).T
        dist_not = np.subtract.outer(vals[:,0], interpol[:,0]) #Length of the triangle side from the cell to the point with data 
        dist_one = np.subtract.outer(vals[:,1], interpol[:,1]) #Length of the triangle side from the cell to the point with data 
        distance_matrix = np.hypot(dist_not,dist_one) #euclidean distance, getting the hypotenuse
        
        weights = 1/(distance_matrix**d) #what if distance is 0 --> np.inf? have to account for the pixel underneath
        weights[np.where(np.isinf(weights))] = 1/(1.0E-50) #Making sure to assign the value of the weather station above the pixel directly to the pixel underneath
        weights /= weights.sum(axis = 0) 

        Zi = np.dot(weights.T, z)
        idw_grid = Zi.reshape(num_row,num_col)

        #Delete at a certain point
        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)

        interpolated_val = idw_grid[y_orig][x_orig] 

        original_val = Cvar_dict[station_name]
        absolute_error = abs(interpolated_val-original_val)
        absolute_error_dictionary[station_name_hold_back] = absolute_error


     return absolute_error_dictionary

In [25]:
def IDEW(latlon_dict,Cvar_dict,input_date,var_name,shapefile,show,file_path_elev,idx_list,d):
    '''Inverse distance elevation weighting
    Parameters
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        input_date (str): the date you want to interpolate for 
        shapefile (str): path to the study area shapefile 
        show (bool): whether you want to plot a map 
        file_path_elev (str): file path to the elevation lookup file 
        idx_list (list): the index of the elevation data column in the lookup file 
        d (int): the weighting function for IDW interpolation 
    Returns 
        idew_grid (np_array): the array of values for the interpolated surface
        maxmin: the bounds of the array surface, for use in other functions 
        elev_array (np_array): the array of elevation values for the study area, so we can return it
        to the cross-validation function for faster processing 
        
    '''
    #Input: lat lon of station, variable (start day, rainfall, etc), date of interest,variable name (for plotting), show (bool true/false), file path to elevation lookup file
    #idx_list (for the column containing the elevation data), d is the power applied to get the weight 
    lat = [] #Initialize empty lists to store data 
    lon = []
    Cvar = []
    for station_name in Cvar_dict.keys(): #Loop through the list of stations 
        if station_name in latlon_dict.keys(): #Make sure the station is present in the latlon dict 
            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            cvar_val = Cvar_dict[station_name]
            lat.append(float(latitude))
            lon.append(float(longitude))
            Cvar.append(cvar_val)
    y = np.array(lat) #Convert to a numpy array for faster processing speed 
    x = np.array(lon)
    z = np.array(Cvar) 

    na_map = gpd.read_file(shapefile)
    bounds = na_map.bounds #Get the bounding box of the shapefile 
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']
    pixelHeight = 10000 #We want a 10 by 10 pixel, or as close as we can get 
    pixelWidth = 10000
            
    num_col = int((xmax - xmin) / pixelHeight) #Calculate the number of rows cols to fill the bounding box at that resolution 
    num_row = int((ymax - ymin) / pixelWidth)


    #We need to project to a projected system before making distance matrix
    source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume NAD83
    xProj, yProj = pyproj.Proj('esri:102001')(x,y) #Convert to Canada Albers Equal Area 

    yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']]) #Add the bounding box coords to the dataset so we can extrapolate the interpolation to cover whole area
    xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

    Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row) #Get the value for lat lon in each cell we just made 
    Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

    Xi,Yi = np.meshgrid(Xi,Yi) #Make a rectangular grid (because eventually we will map the values)
    Xi,Yi = Xi.flatten(), Yi.flatten() #Then we flatten the arrays for easier processing 
    maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)] #We will later return this for use in other functions 

    vals = np.vstack((xProj,yProj)).T #vertically stack station x and y vals and then transpose them so they are in pairs 

    interpol = np.vstack((Xi,Yi)).T #Do the same thing for the grid x and y vals 
    dist_not = np.subtract.outer(vals[:,0], interpol[:,0]) #Length of the triangle side from the cell to the point with data 
    dist_one = np.subtract.outer(vals[:,1], interpol[:,1]) #Length of the triangle side from the cell to the point with data 
    distance_matrix = np.hypot(dist_not,dist_one) #Euclidean distance, getting the hypotenuse


    weights = 1/(distance_matrix**d) #what if distance is 0 --> np.inf? have to account for the pixel underneath
    weights[np.where(np.isinf(weights))] = 1/(1.0E-50) #Making sure to assign the value of the weather station above the pixel directly to the pixel underneath
    weights /= weights.sum(axis = 0) #The weights must add up to 0 


    Zi = np.dot(weights.T, z) #Take the dot product of the weights and the values, in this case the dot product is the sum product over the last axis of Weights.T and z

    idw_grid = Zi.reshape(num_row,num_col) #reshape the array into the proper format for the map 

    #Elevation weights
    #Lon (X) goes in first for a REASON. It has to do with order in the lookup file. 
    concat = np.array((Xi.flatten(), Yi.flatten())).T #Preparing the coordinates to send to the function that will get the elevation grid 
    send_to_list = concat.tolist()
    send_to_tuple = [tuple(x) for x in send_to_list] #The elevation function takes a tuple 


    Xi1_grd=[]
    Yi1_grd=[]
    elev_grd = []
    elev_grd_dict = finding_data_frm_lookup(send_to_tuple,file_path_elev,idx_list) #Get the elevations from the lookup file 

    for keys in elev_grd_dict.keys(): #The keys are each lat lon pair 
        x= keys[0]
        y = keys[1]
        Xi1_grd.append(x)
        Yi1_grd.append(y)
        elev_grd.append(elev_grd_dict[keys]) #Append the elevation data to the empty list 

    elev_array = np.array(elev_grd) #make an elevation array

    

    elev_dict= finding_data_frm_lookup(zip(xProj, yProj),file_path_elev,idx_list) #Get the elevations for the stations 

    xProj_input=[]
    yProj_input=[]
    e_input = []


    for keys in zip(xProj,yProj): #Repeat process for just the stations not the whole grid 
        x= keys[0]
        y = keys[1]
        xProj_input.append(x)
        yProj_input.append(y)
        e_input.append(elev_dict[keys])

    source_elev = np.array(e_input)


    vals2 = np.vstack(source_elev).T

    interpol2 = np.vstack(elev_array).T

    dist_not2 = np.subtract.outer(vals2[0], interpol2[0]) #Get distance in terms of the elevation (vertical distance) from the station to the point to be interpolated 
    dist_not2 = np.absolute(dist_not2) #Take the absolute value, we just care about what is the difference 
    weights2 = 1/(dist_not2**d) #Get the inverse distance weight 
    weights2[np.where(np.isinf(weights2))] = 1 #In the case of no elevation change 
    weights2 /= weights2.sum(axis = 0) #Make weights add up to 1 


    fin = 0.8*np.dot(weights.T,z) + 0.2*np.dot(weights2.T,z) #Weight distance as 0.8 and elevation as 0.2 

    idew_grid = fin.reshape(num_row,num_col) #Reshape the final array 


    if show: #Plot if show == True 
        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(idew_grid,extent=(xProj_extent.min()-1,xProj_extent.max()+1,yProj_extent.max()-1,yProj_extent.min()+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.scatter(xProj,yProj,c=z,edgecolors='k')

        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label(var_name) 
        
        title = 'IDEW Interpolation for %s on %s'%(var_name,input_date)
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude') 

        plt.show()


    return idew_grid, maxmin, elev_array

In [27]:
def cross_validate_IDEW(latlon_dict,Cvar_dict,shapefile,file_path_elev,elev_array,idx_list,d):
    '''Leave-one-out cross-validation procedure for IDEW
    Parameters
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        shapefile (str): path to the study area shapefile 
        file_path_elev (str): file path to the elevation lookup file 
        elev_array (np_array): the elevation array for the study area 
        idx_list (list): the index of the elevation data column in the lookup file 
        d (int): the weighting function for IDW interpolation 
    Returns 
        absolute_error_dictionary (dict): a dictionary of the absolute error at each station when it
        was left out 
    '''
    x_origin_list = []
    y_origin_list = [] 

    absolute_error_dictionary = {} #for plotting
    station_name_list = []
    projected_lat_lon = {}

    for station_name in Cvar_dict.keys():
        if station_name in latlon_dict.keys():
            station_name_list.append(station_name)

            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
            Plat = float(Plat)
            Plon = float(Plon)
            projected_lat_lon[station_name] = [Plat,Plon]



    for station_name_hold_back in station_name_list:

        lat = []
        lon = []
        Cvar = []
        for station_name in sorted(Cvar_dict.keys()):
            if station_name in latlon_dict.keys():
                if station_name != station_name_hold_back:
                    loc = latlon_dict[station_name]
                    latitude = loc[0]
                    longitude = loc[1]
                    cvar_val = Cvar_dict[station_name]
                    lat.append(float(latitude))
                    lon.append(float(longitude))
                    Cvar.append(cvar_val)
                else:

                    pass
                
        y = np.array(lat)
        x = np.array(lon)
        z = np.array(Cvar) #what if we add the bounding locations to the array??? ==> that would be extrapolation not interpolation? 

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds
        xmax = bounds['maxx']
        xmin= bounds['minx']
        ymax = bounds['maxy']
        ymin = bounds['miny']
        pixelHeight = 10000 
        pixelWidth = 10000
                
        num_col = int((xmax - xmin) / pixelHeight)
        num_row = int((ymax - ymin) / pixelWidth)


        #We need to project to a projected system before making distance matrix
        source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
        xProj, yProj = pyproj.Proj('esri:102001')(x,y)

        yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
        xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

        Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
        Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

        Xi,Yi = np.meshgrid(Xi,Yi)
        Xi,Yi = Xi.flatten(), Yi.flatten()
        maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)]

        vals = np.vstack((xProj,yProj)).T
        
        interpol = np.vstack((Xi,Yi)).T
        dist_not = np.subtract.outer(vals[:,0], interpol[:,0]) #Length of the triangle side from the cell to the point with data 
        dist_one = np.subtract.outer(vals[:,1], interpol[:,1]) #Length of the triangle side from the cell to the point with data 
        distance_matrix = np.hypot(dist_not,dist_one) #euclidean distance, getting the hypotenuse
        
        weights = 1/(distance_matrix**d) #what if distance is 0 --> np.inf? have to account for the pixel underneath
        weights[np.where(np.isinf(weights))] = 1/(1.0E-50) #Making sure to assign the value of the weather station above the pixel directly to the pixel underneath
        weights /= weights.sum(axis = 0) 

        Zi = np.dot(weights.T, z)
        idw_grid = Zi.reshape(num_row,num_col)


        
        elev_dict= GD.finding_data_frm_lookup(zip(yProj, xProj),file_path_elev,idx_list)

        xProj_input=[]
        yProj_input=[]
        e_input = []
        

        for keys in zip(yProj,xProj): # in case there are two stations at the same lat\lon 
            x= keys[0]
            y = keys[1]
            xProj_input.append(x)
            yProj_input.append(y)
            e_input.append(elev_dict[keys])

        source_elev = np.array(e_input)


        vals2 = np.vstack(source_elev).T

        interpol2 = np.vstack(elev_array).T

        dist_not2 = np.subtract.outer(vals2[0], interpol2[0])
        dist_not2 = np.absolute(dist_not2)
        weights2 = 1/(dist_not2**d)

        weights2[np.where(np.isinf(weights2))] = 1 
        weights2 /= weights2.sum(axis = 0)

        fin = 0.8*np.dot(weights.T,z) + 0.2*np.dot(weights2.T,z)
        
        fin = fin.reshape(num_row,num_col)

        #Calc the RMSE, MAE at the pixel loc
        #Delete at a certain point
        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)

        interpolated_val = fin[y_orig][x_orig] 

        original_val = Cvar_dict[station_name]
        absolute_error = abs(interpolated_val-original_val)
        absolute_error_dictionary[station_name_hold_back] = absolute_error


    return absolute_error_dictionary

In [28]:
import scipy.interpolate as interpolate
import math

In [29]:
def TPS(latlon_dict,Cvar_dict,input_date,var_name,shapefile,show,phi):
    '''Thin plate splines interpolation implemented using the interpolate radial basis function from 
    SciPy 
    Parameters
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        input_date (str): the date you want to interpolate for 
        var_name (str): name of the variable you are interpolating
        shapefile (str): path to the study area shapefile 
        show (bool): whether you want to plot a map 
        phi (float): smoothing parameter for the thin plate spline, if 0 no smoothing 
    Returns 
        spline (np_array): the array of values for the interpolated surface
        maxmin: the bounds of the array surface, for use in other functions 
    
    '''
    x_origin_list = []
    y_origin_list = []
    z_origin_list = [] 

    absolute_error_dictionary = {} #for plotting
    station_name_list = []
    projected_lat_lon = {}

    for station_name in Cvar_dict.keys():
        if station_name in latlon_dict.keys():
            station_name_list.append(station_name)

            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
            Plat = float(Plat)
            Plon = float(Plon)
            projected_lat_lon[station_name] = [Plat,Plon]

    lat = []
    lon = []
    Cvar = []
    for station_name in Cvar_dict.keys(): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step
        if station_name in latlon_dict.keys():
            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            cvar_val = Cvar_dict[station_name]
            lat.append(float(latitude))
            lon.append(float(longitude))
            Cvar.append(cvar_val)
    y = np.array(lat)
    x = np.array(lon)
    z = np.array(Cvar)

    Cvar = []
    for station_name in Cvar_dict.keys(): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step 

        cvar_val = Cvar_dict[station_name]

        Cvar.append(cvar_val)

    z2 = np.array(Cvar)


    #for station_name in sorted(Cvar_dict.keys()): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step
    for station_name_hold_back in station_name_list:

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds

        pixelHeight = 10000 
        pixelWidth = 10000


        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)
        z_origin_list.append(Cvar_dict[station_name_hold_back])


    na_map = gpd.read_file(shapefile)
    bounds = na_map.bounds
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']
    pixelHeight = 10000 
    pixelWidth = 10000
            
    num_col = int((xmax - xmin) / pixelHeight)
    num_row = int((ymax - ymin) / pixelWidth)
    

    source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
    xProj, yProj = pyproj.Proj('esri:102001')(x,y)

    yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
    xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

    maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)]

    Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
    Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

    Xi,Yi = np.meshgrid(Xi,Yi)

    empty_grid = np.empty((num_row,num_col,))*np.nan

    for x,y,z in zip(x_origin_list,y_origin_list,z_origin_list):
        empty_grid[y][x] = z



    vals = ~np.isnan(empty_grid)

    func = interpolate.Rbf(Xi[vals],Yi[vals],empty_grid[vals], function='thin_plate',smooth=phi)
    thin_plate = func(Xi,Yi)
    spline = thin_plate.reshape(num_row,num_col)

    if show: 

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(spline,extent=(xProj_extent.min()-1,xProj_extent.max()+1,yProj_extent.max()-1,yProj_extent.min()+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.scatter(xProj,yProj,c=z_origin_list,edgecolors='k',linewidth=1)

        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label(var_name) 
        
        title = 'Thin Plate Spline Interpolation for %s on %s'%(var_name,input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude') 

        plt.show()

    return spline, maxmin

In [31]:
def cross_validate_tps(latlon_dict,Cvar_dict,shapefile,phi):
    '''Leave-one-out cross-validation for thin plate splines 
    Parameters
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        shapefile (str): path to the study area shapefile 
        phi (float): smoothing parameter for the thin plate spline, if 0 no smoothing 
    Returns 
        absolute_error_dictionary (dict): a dictionary of the absolute error at each station when it
        was left out 
     '''
    x_origin_list = []
    y_origin_list = [] 
    z_origin_list = []
    absolute_error_dictionary = {} #for plotting
    station_name_list = []
    projected_lat_lon = {}

    for station_name in Cvar_dict.keys():
        if station_name in latlon_dict.keys():
            station_name_list.append(station_name)

            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
            Plat = float(Plat)
            Plon = float(Plon)
            projected_lat_lon[station_name] = [Plat,Plon]


    for station_name_hold_back in station_name_list:

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds

        pixelHeight = 10000 
        pixelWidth = 10000


        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)
        z_origin_list.append(Cvar_dict[station_name_hold_back])


    for station_name_hold_back in station_name_list:

        lat = []
        lon = []
        Cvar = []
        for station_name in sorted(Cvar_dict.keys()):
            if station_name in latlon_dict.keys():
                if station_name != station_name_hold_back:
                    loc = latlon_dict[station_name]
                    latitude = loc[0]
                    longitude = loc[1]
                    cvar_val = Cvar_dict[station_name]
                    lat.append(float(latitude))
                    lon.append(float(longitude))
                    Cvar.append(cvar_val)
                else:
                    pass
                    
        y = np.array(lat)
        x = np.array(lon)
        z = np.array(Cvar) #what if we add the bounding locations to the array??? ==> that would be extrapolation not interpolation? 

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds
        xmax = bounds['maxx']
        xmin= bounds['minx']
        ymax = bounds['maxy']
        ymin = bounds['miny']
        pixelHeight = 10000 
        pixelWidth = 10000
                
        num_col = int((xmax - xmin) / pixelHeight)
        num_row = int((ymax - ymin) / pixelWidth)


        #We need to project to a projected system before making distance matrix
        source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
        xProj, yProj = pyproj.Proj('esri:102001')(x,y)

        yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
        xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

        Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
        Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

        Xi,Yi = np.meshgrid(Xi,Yi)

        empty_grid = np.empty((num_row,num_col,))*np.nan

        for x,y,z in zip(x_origin_list,y_origin_list,z_origin_list):
            empty_grid[y][x] = z



        vals = ~np.isnan(empty_grid)

        func = interpolate.Rbf(Xi[vals],Yi[vals],empty_grid[vals], function='thin_plate',smooth=phi)
        thin_plate = func(Xi,Yi)
        spline = thin_plate.reshape(num_row,num_col)

        #Calc the RMSE, MAE, NSE, and MRAE at the pixel loc
        #Delete at a certain point
        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)

        interpolated_val = spline[y_orig][x_orig] 

        original_val = Cvar_dict[station_name]
        absolute_error = abs(interpolated_val-original_val)
        absolute_error_dictionary[station_name_hold_back] = absolute_error

    return absolute_error_dictionary



In [32]:
from pykrige.ok import OrdinaryKriging

In [33]:
def OKriging(latlon_dict,Cvar_dict,input_date,var_name,shapefile,model,show):
    '''Implement ordinary kriging 
    Parameters
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        input_date (str): the date you want to interpolate for 
        var_name (str): name of the variable you are interpolating
        shapefile (str): path to the study area shapefile 
        model (str): semivariogram model name, ex. 'gaussian'
        show (bool): whether you want to plot a map 
    Returns 
        kriging_surface (np_array): the array of values for the interpolated surface
        maxmin: the bounds of the array surface, for use in other functions 
        
        if there is an error when interpolating the surface, it does not return anything 
        
    '''
    x_origin_list = []
    y_origin_list = []
    z_origin_list = [] 

    absolute_error_dictionary = {} #for plotting
    station_name_list = []
    projected_lat_lon = {}

    for station_name in Cvar_dict.keys():
        station_name_list.append(station_name)

        loc = latlon_dict[station_name]
        latitude = loc[0]
        longitude = loc[1]
        Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
        Plat = float(Plat)
        Plon = float(Plon)
        projected_lat_lon[station_name] = [Plat,Plon]

    lat = []
    lon = []
    Cvar = []
    for station_name in Cvar_dict.keys(): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step 
        loc = latlon_dict[station_name]
        latitude = loc[0]
        longitude = loc[1]
        cvar_val = Cvar_dict[station_name]
        lat.append(float(latitude))
        lon.append(float(longitude))
        Cvar.append(cvar_val)
    y = np.array(lat)
    x = np.array(lon)
    z = np.array(Cvar)

    Cvar = []
    for station_name in Cvar_dict.keys(): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step 

        cvar_val = Cvar_dict[station_name]

        Cvar.append(cvar_val)

    z2 = np.array(Cvar)


    #for station_name in sorted(Cvar_dict.keys()): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step
    for station_name_hold_back in station_name_list:

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds

        pixelHeight = 10000 
        pixelWidth = 10000


        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)
        z_origin_list.append(Cvar_dict[station_name_hold_back])


    #Uncomment to check if the correct stations are being accessed
    na_map = gpd.read_file(shapefile)
    bounds = na_map.bounds
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']
    
    pixelHeight = 10000 
    pixelWidth = 10000
                
    num_col = int((xmax - xmin) / pixelHeight)
    num_row = int((ymax - ymin) / pixelWidth)
    

    source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
    xProj, yProj = pyproj.Proj('esri:102001')(x,y)

    yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
    xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

    maxmin = [np.min(yProj_extent),np.max(yProj_extent),np.max(xProj_extent),np.min(xProj_extent)]

    Yi1 = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
    Xi1 = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

    Xi,Yi = np.meshgrid(Xi1,Yi1)

    empty_grid = np.empty((num_row,num_col,))*np.nan

    for x3,y3,z3 in zip(x_origin_list,y_origin_list,z_origin_list):
        empty_grid[y3][x3] = z3



    vals = ~np.isnan(empty_grid)



    OK = OrdinaryKriging(xProj,yProj,z,variogram_model=model,verbose=False,enable_plotting=False)
    try: 
        z1,ss1 = OK.execute('grid',Xi1,Yi1,n_closest_points=10,backend='C') #n_closest_points=10
        
        kriging_surface = z1.reshape(num_row,num_col)
        if show:
            fig, ax = plt.subplots(figsize= (15,15))
            crs = {'init': 'esri:102001'}

            na_map = gpd.read_file(shapefile)
            
          
            plt.imshow(kriging_surface,extent=(xProj_extent.min()-1,xProj_extent.max()+1,yProj_extent.max()-1,yProj_extent.min()+1)) 
            na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
                
            plt.scatter(xProj,yProj,c=z_origin_list,edgecolors='k',linewidth=1)

            plt.gca().invert_yaxis()
            cbar = plt.colorbar()
            cbar.set_label(var_name) 
            
            title = 'Ordinary Kriging Interpolation for %s on %s'%(var_name,input_date) 
            fig.suptitle(title, fontsize=14)
            plt.xlabel('Longitude')
            plt.ylabel('Latitude') 

            plt.show()

        return kriging_surface, maxmin

    except:
        pass 


In [35]:
def cross_validate_OK(latlon_dict,Cvar_dict,shapefile,model):
    '''Cross_validate the ordinary kriging 
    Parameters 
        latlon_dict (dict): the latitude and longitudes of the hourly stations, loaded from the 
        .json file
        Cvar_dict (dict): dictionary of weather variable values for each station 
        shapefile (str): path to the study area shapefile 
        model (str): semivariogram model name, ex. 'gaussian'
    Returns 
        absolute_error_dictionary (dict): a dictionary of the absolute error at each station when it
        was left out 
    '''
    x_origin_list = []
    y_origin_list = [] 
    z_origin_list = []
    absolute_error_dictionary = {} #for plotting
    station_name_list = []
    projected_lat_lon = {}

    for station_name in Cvar_dict.keys():
        if station_name in latlon_dict.keys():
            station_name_list.append(station_name)

            loc = latlon_dict[station_name]
            latitude = loc[0]
            longitude = loc[1]
            Plat, Plon = pyproj.Proj('esri:102001')(longitude,latitude)
            Plat = float(Plat)
            Plon = float(Plon)
            projected_lat_lon[station_name] = [Plat,Plon]



    for station_name_hold_back in station_name_list:

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds

        pixelHeight = 10000 
        pixelWidth = 10000


        coord_pair = projected_lat_lon[station_name_hold_back]

        x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
        y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
        x_origin_list.append(x_orig)
        y_origin_list.append(y_orig)
        z_origin_list.append(Cvar_dict[station_name_hold_back])




    #for station_name in sorted(Cvar_dict.keys()): #DONT use list of stations, because if there's a no data we delete that in the climate dictionary step
    for station_name_hold_back in station_name_list:
         
        lat = []
        lon = []
        Cvar = []
        for station_name in sorted(Cvar_dict.keys()):
            if station_name in latlon_dict.keys():
                if station_name != station_name_hold_back:
                    loc = latlon_dict[station_name]
                    latitude = loc[0]
                    longitude = loc[1]
                    cvar_val = Cvar_dict[station_name]
                    lat.append(float(latitude))
                    lon.append(float(longitude))
                    Cvar.append(cvar_val)
                else:
                    #print('Skipping station!')
                    pass
                
        y = np.array(lat)
        x = np.array(lon)
        z = np.array(Cvar) #what if we add the bounding locations to the array??? ==> that would be extrapolation not interpolation? 

        na_map = gpd.read_file(shapefile)
        bounds = na_map.bounds
        xmax = bounds['maxx']
        xmin= bounds['minx']
        ymax = bounds['maxy']
        ymin = bounds['miny']
        pixelHeight = 10000 
        pixelWidth = 10000
                
        num_col = int((xmax - xmin) / pixelHeight)
        num_row = int((ymax - ymin) / pixelWidth)


        #We need to project to a projected system before making distance matrix
        source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume 
        xProj, yProj = pyproj.Proj('esri:102001')(x,y)

        yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']])
        xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])

        Yi1 = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row)
        Xi1 = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

        Xi,Yi = np.meshgrid(Xi1,Yi1)
        
        empty_grid = np.empty((num_row,num_col,))*np.nan

        for x3,y3,z3 in zip(x_origin_list,y_origin_list,z_origin_list):
            empty_grid[y3][x3] = z3



        vals = ~np.isnan(empty_grid)

        OK = OrdinaryKriging(xProj,yProj,z,variogram_model=model,verbose=False,enable_plotting=False)
        try: 
            z1,ss1 = OK.execute('grid',Xi1,Yi1,n_closest_points=10,backend='C') #n_closest_points=10
    
            kriging_surface = z1.reshape(num_row,num_col)

        #Calc the RMSE, MAE at the pixel loc
        #Delete at a certain point
            coord_pair = projected_lat_lon[station_name_hold_back]

            x_orig = int((coord_pair[0] - float(bounds['minx']))/pixelHeight) #lon 
            y_orig = int((coord_pair[1] - float(bounds['miny']))/pixelWidth) #lat
            x_origin_list.append(x_orig)
            y_origin_list.append(y_orig)

            interpolated_val = kriging_surface[y_orig][x_orig] #which comes first?

            original_val = Cvar_dict[station_name]
            absolute_error = abs(interpolated_val-original_val)
            absolute_error_dictionary[station_name_hold_back] = absolute_error
        except:
            pass 


    return absolute_error_dictionary 

The next group of functions is for calculating the FWI metrics. Much of this code is translated directly from the R cffdrs package, see: 

Wang, X., Wotton, B. M., Cantin, A. S., Parisien, M. A., Anderson, K., Moore, B., & Flannigan, M. D. (2017). cffdrs: an R package for the Canadian Forest Fire Danger Rating System. Ecological Processes, 6(1). https://doi.org/10.1186/s13717-017-0070-z

In [36]:
%%cython

from datetime import datetime, timedelta, date

def get_date_index(str year,str input_date,int month):
    '''Get the number of days for the date of interest from the first of the month of interest
    Example, convert to days since March 1 
    Parameters
        year (str): year of interest 
        input_date (str): input date of interest
        month (str): the month from when you want to calculate the date (ex, Mar 1)
    Returns 
        day (int): days since 1st of the month of interest 
    '''
    d0 = date(int(year), month, 1)
    input_date = str(input_date)
    d1 = date(int(input_date[0:4]), int(input_date[5:7]), int(input_date[8:10])) #convert to days since march 1/oct 1 so we can interpolate
    delta = d1 - d0
    day = int(delta.days)
    return day 

In [37]:
%%cython

import numpy as np

def make_start_date_mask(int day_index,day_interpolated_surface):
    '''Turn the interpolated surface of start dates into a numpy array
    Parameters
        day_index (int): index of the day of interest since Mar 1
        day_interpolated_surface (np_array): the interpolated surface of the start dates across the 
        study area 
    Returns 
        new (np_array): a mask array of the start date, either activated (1) or inactivated (np.nan)
    '''
    shape = day_interpolated_surface.shape
    new = np.ones(shape)
    new[day_interpolated_surface <= day_index] = 1 #If the day in the interpolated surface is before the index, it is activated 
    new[day_interpolated_surface > day_index] = np.nan #If it is the opposite it will be masked out, so assign it to np.nan (no data) 
    return new

def make_end_date_mask(int day_index,day_interpolated_surface):
    '''Turn the interpolated surface of end dates into a numpy array
    Parameters
        day_index (int): index of the day of interest since Oct 1
        day_interpolated_surface (np_array): the interpolated surface of the end dates across the 
        study area 
    Returns 
        new (np_array): a mask array of the end date, either activated (1) or inactivated (np.nan)
    '''
    shape = day_interpolated_surface.shape
    new = np.ones(shape)
    new[day_interpolated_surface <= day_index] = np.nan #If the day in the interpolated surface is before the index, its closed
    new[day_interpolated_surface > day_index] = 1 #If it is the opposite it will be left open, so assign it to 1
    return new

Next, we will calculate whether we need the overwinter procedure for each cell in the study area. These are areas where winter precipitation is less than 200 mm (Lawson & Armitage, 2008). 

In [38]:
import matplotlib.pyplot as plt


def get_overwinter_pcp(overwinter_dates, file_path_daily,start_surface,end_surface,maxmin, shapefile,
                       show,date_dictionary,latlon_dictionary, file_path_elev,idx_list, json):
    '''Get the total amount of overwinter pcp for the purpose of knowing where to use the 
    overwinter DC procedure 
    Parameters
        overwinter_dates (list): list of dates that are in the winter (i.e. station shut down to 
        start up), you can just input generally Oct 1-June 1 and stations that are still active 
        will be masked out 
        file_path_daily (str): file path to the daily feather files containing the precipitation data
        start_surface (np_array): array containing the interpolated start-up date for each cell 
        end_surface (np_array): array containing the interpolated end date for each cell, from the 
        year before 
        maxmin (list): bounds of the study area 
        shapefile (str): path to the study area shapefile 
        date_dictionary (dict, loaded from .json): lookup file that has what day/month pairs each 
        station contains data for 
        latlon_dictionary (dict, loaded from .json): lat lons of the daily stations
        file_path_elev (str): file path to the elevation lookup file 
        idx_list (list): the index of the elevation data column in the lookup file 
        json (bool): if True, convert the array to a flat list so it can be written as a .json file
        to the hard drive
    Returns 
        pcp_overwinter (np_array): array of interpolated overwinter precipitation for the study area 
        overwinter_reqd (np_array): array indicating where to do the overwinter DC procedure 
    '''
    pcp_list = [] 

    for o_dat in overwinter_dates: #dates is from Oct 1 year before to start day current year (up to Apr 1)
        year = str(o_dat)[0:4]
        index = overwinter_dates.index(o_dat) #we need to take index while its still a timestamp before convert to str
        o_dat = str(o_dat)
        day_index= get_date_index(year,o_dat,3)
        eDay_index = get_date_index(year,o_dat,10)
        if int(str(o_dat)[5:7]) >= 10:


            invertEnd = np.ones(end_surface.shape)
            endMask = make_end_date_mask(eDay_index,end_surface)
            invertEnd[endMask == 1] = 0 #0 in place of np.nan, because we use sum function later 
            invertEnd[np.where(np.isnan(endMask))] = 1

            endMask = invertEnd 
            mask = np.ones(start_surface.shape) #No stations will start up until the next spring
        elif int(str(o_dat)[5:7]) < 10 and int(str(o_dat)[5:7]) >= 7: #All stations are in summer

            endMask = np.zeros(end_surface.shape)
            mask = np.zeros(start_surface.shape)
        else:

            endMask = np.ones(end_surface.shape) #by this time (Jan 1) all stations are in winter
            invertMask = np.ones(start_surface.shape)
            mask = make_start_date_mask(day_index,start_surface)
            invertMask[mask == 1] = 0 #If the station has started, stop counting it 
            invertMask[np.where(np.isnan(mask))] = 1 #If the station is still closed, keep counting
            mask = invertMask

        rainfall = get_pcp(str(o_dat)[0:10],file_path_daily,date_dictionary)
        rain_grid,maxmin = IDW(latlon_dictionary,rainfall,o_dat,'Precipitation',shapefile,False,1)

        masked = rain_grid * mask * endMask

        masked[np.where(np.isnan(masked))] = 0 #sum can't handle np.nan

        pcp_list.append(masked)

        
    #when we sum, we need to treat nan as 0, otherwise any nan in the list will cause the whole val to be nan 
    pcp_overwinter = sum(pcp_list)


    overwinter_reqd = np.ones(pcp_overwinter.shape)
    overwinter_reqd[pcp_overwinter> 200] = np.nan #not Required
    overwinter_reqd[pcp_overwinter <= 200] = 1 #required 

    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(overwinter_reqd,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        
        title = 'Areas Requiring Overwinter DC Procedure for %s'%(year)
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()

    if json:
        pcp_overwinter = list(pcp_overwinter.flatten()) #json cannot handle numpy arrays 
        overwinter_reqd = list(overwinter_reqd.flatten()) 

    return pcp_overwinter,overwinter_reqd #This is the array, This is a mask 

In [40]:
from descartes import PolygonPatch

In [41]:
def DC(input_date,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,dc_yesterday,index,show,shapefile,mask,endMask,
       last_DC_val_before_shutdown,overwinter):
    '''Calculate the DC. See cffdrs R code
    Parameters
        input_date (str): input date of interest
        rain_grid: interpolated surface for rainfall on the date of interest
        temp_grid: interpolated surface for temperature on the date of interest
        wind_grid: interpolated surface for wind on the date of interest
        maxmin: bounds of the study area 
        dc_yesterday: array of DC values for yesterday (from the dc stack list/if this function
        is being used inside dc_stack it is calculated then) 
        index (int): index of the date since Mar 1
        show (bool): whether you want to show the map 
        shapefile (str): path to the study area shapefile
        mask (np_array): mask for the start dates 
        endMask (np_array): mask for the end days 
        last_DC_val_before_shutdown (np_array): array for last dc values before cell shut down, if no areas 
        required the procedure, you can input an empty array of the correct size (if not using overwinter, input
        the empty array)
        overwinter (bool): whether or not to implement the overwinter procedure 
    Returns 
        dc1 (np_array): array of dc values on the date on interest for the study area
    '''
    
    yesterday_index = index-1
    if yesterday_index == -1:
        if overwinter:
            rain_shape = rain_grid.shape
            dc_initialize = np.zeros(rain_shape)
            dc_initialize[np.isnan(last_DC_val_before_shutdown)] = 15
            dc_initialize[~np.isnan(last_DC_val_before_shutdown)] = last_DC_val_before_shutdown[~np.isnan(last_DC_val_before_shutdown)]
            dc_yesterday1 = dc_initialize*mask 
        else: 
            rain_shape = rain_grid.shape
            dc_initialize = np.zeros(rain_shape)+15
            dc_yesterday1 = dc_initialize
            dc_yesterday1 = dc_yesterday1*mask #mask out areas that haven't started
    else:
        if overwinter:
            dc_yesterday1 = dc_yesterday
            dc_yesterday1[np.where(np.isnan(dc_yesterday1) & ~np.isnan(mask) & ~np.isnan(last_DC_val_before_shutdown))] = last_DC_val_before_shutdown[np.where(np.isnan(dc_yesterday1) & ~np.isnan(mask) & ~np.isnan(last_DC_val_before_shutdown))]
            dc_yesterday1[np.where(np.isnan(dc_yesterday1) & ~np.isnan(mask) & np.isnan(last_DC_val_before_shutdown))] = 15
        else: 
            dc_yesterday1 = dc_yesterday
            dc_yesterday1[np.where(np.isnan(dc_yesterday1) & ~np.isnan(mask))] = 15 #set started pixels since yesterday to 15

    input_date = str(input_date)
    month = int(input_date[6])
    #Get day length factor

    f101 = [-1.6,-1.6,-1.6,0.9,3.8,5.8,6.4,5,2.4,0.4,-1.6,-1.6]

    #Put constraint on low end of temp
    temp_grid[temp_grid < -2.8] = -2.8

    #E22 potential evapT

    pe = (0.36*(temp_grid+2.8)+f101[month])/2

    #Make empty dmc array
    new_shape = dc_yesterday1.shape
    dc = np.zeros(new_shape)

    #starting rain

    netRain = 0.83*rain_grid-1.27

    #eq 19
    smi = 800*np.exp(-1*dc_yesterday1/400)

    #eq 21
    dr0 = dc_yesterday1 -400*np.log(1+3.937*netRain/smi) #log is the natural logarithm 
    dr0[dr0<0] = 0
    dr0[rain_grid <= 2.8] = dc_yesterday1[rain_grid <= 2.8]

    dc1 = dr0 + pe
    dc1[dc1 < 0] = 0
        
    dc1 = dc1 * mask * endMask
    if show == True: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        circ = PolygonPatch(na_map['geometry'][0],visible=False)
        ax.add_patch(circ) 
        plt.imshow(dc1,extent=(xProj_extent.min(),xProj_extent.max(),yProj_extent.max(),yProj_extent.min()),clip_path=circ, 
                   clip_on=True) 
      
        #plt.imshow(dc1,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('DC') 
        
        title = 'DC for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()

    return dc1


In [42]:
def dc_stack(dates,file_path_daily,file_path_hourly,var_name,shapefile,day_interpolated_surface,end_interpolated_surface,
             last_DC_val_before_shutdown,overwinter,file_path_elev,idx_list,date_dictionary,latlon_dict,latlon_dictionary,
             json,interpolation_method):
    '''Calc dc for each day in season. This is the only metric with overwinter procedure applied. 
    For notes see cffdrs R code.
    Parameters
        dates (list): list of all dates within the fire season, inactive stations will be masked out 
        so you can define it as Mar 1 - Dec 31
        file_path_daily (str): file path to the daily feather files
        file_path_hourly (str): file path to the hourly feather files
        var_name (str): name of the variable you are interpolating
        shapefile (str): path to the study area shapefile 
        day_interpolated_surface (np_array): array of start-up days (since Mar 1) for the study area
        end_interpolated_surface (np_array): array of end days (since Oct 1) for the study area
        last_DC_val_before_shutdown (np_array): the last DC value in each cell before that cell shut down
        for the winter; only need to input this if overwinter = True otherwise you can just input None 
        overwinter (bool): if True, the program overwinters the DC where it needs to be
        file_path_elev (str): file path to the elevation lookup file 
        idx_list (list): the index of the elevation data column in the lookup file 
        date_dictionary (dict, loaded from .json): lookup file that has what day/month pairs each 
        station contains data for 
        latlon_dict (dict, loaded from .json): lat lons of the hourly stations
        latlon_dictionary (dict, loaded from .json): lat lons of the daily stations
        json (bool): if True, convert the array to a flat list so it can be written as a .json file
        interpolation_method (str): the interpolation method to use to get the continuous DC surface, 
        there are seven options - IDW-1, IDW-2, IDEW-1, IDEW-2, TPS, TPSS, OK 
    Returns 
        dc_list (list of np_array): a list of the interpolated surfaces for the drought code for 
        each day in the fire season 
    '''

    print(interpolation_method)
    
    dc_list = [] 
    count = 0 
    for dat in dates:
        gc.collect() 
        year = str(dat)[0:4]
        index = dates.index(dat)
        dat = str(dat)
        day_index= get_date_index(year,dat,3)
        eDay_index = get_date_index(year,dat,10)

        mask1 = make_start_date_mask(day_index,day_interpolated_surface)
        if eDay_index < 0:
            endMask = np.ones(end_interpolated_surface.shape) #in the case that the index is before Oct 1
        else: 
            endMask = make_end_date_mask(eDay_index,end_interpolated_surface)

        hourly = str(dat)[0:10]+' 13:00'
        
        rainfall = get_pcp(str(dat)[0:10],file_path_daily,date_dictionary)
        wind = get_wind_speed(hourly,file_path_hourly) #Using the list, get the data for wind speed for those stations on the input date
        temp = get_noon_temp(hourly,file_path_hourly) #Using the list, get the data for temperature for those stations on the input date
        rh =get_relative_humidity(hourly,file_path_hourly) #Using the list, get the data for rh% for those stations on the input date

        #what type of interpolation are we using here?

        if interpolation_method == 'IDW-1': 

        
            rain_grid, maxmin = IDW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,1)
            temp_grid, maxmin = IDW(latlon_dict,temp,hourly,var_name,shapefile,False,1)
            rh_grid, maxmin = IDW(latlon_dict,rh,hourly,var_name,shapefile,False,1)
            wind_grid, maxmin = IDW(latlon_dict,wind,hourly,var_name,shapefile,False,1)

        if interpolation_method == 'IDW-2': 

        
            rain_grid, maxmin = idw.IDW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,2)
            temp_grid, maxmin = idw.IDW(latlon_dict,temp,hourly,var_name,shapefile,False,2)
            rh_grid, maxmin = idw.IDW(latlon_dict,rh,hourly,var_name,shapefile,False,2)
            wind_grid, maxmin = idw.IDW(latlon_dict,wind,hourly,var_name,shapefile,False,2)

        if interpolation_method == 'IDEW-1':

            rain_grid, maxmin, elev_array= IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,1)
            temp_grid, maxmin, elev_array= IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            rh_grid, maxmin, elev_array = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            wind_grid, maxmin, elev_array= IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            
        if interpolation_method == 'IDEW-2':

            rain_grid, maxmin, elev_array = IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,2)
            temp_grid, maxmin, elev_array = IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            rh_grid, maxmin, elev_array = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            wind_grid, maxmin, elev_array = IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)

        if interpolation_method == 'TPS':

            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,0)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,0)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,0)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,0)

        if interpolation_method == 'TPSS':

            num_stations_R = len(rainfall.keys())
            num_stations_t = len(temp.keys())
            num_stations_rh = len(rh.keys())
            num_stations_w = len(wind.keys())
            
            smoothing_parameterR = int(num_stations_R)-(math.sqrt(2*num_stations_R))
            smoothing_parameterT = int(num_stations_t)-(math.sqrt(2*num_stations_t))
            smoothing_parameterRH = int(num_stations_rh)-(math.sqrt(2*num_stations_rh))
            smoothing_parameterW = int(num_stations_w)-(math.sqrt(2*num_stations_w))
            
            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,smoothing_parameterR)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,smoothing_parameterT)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,smoothing_parameterRH)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,smoothing_parameterW)


        if interpolation_method == 'OK':

            rain_grid, maxmin = OKriging(latlon_dictionary,rainfall,dat,var_name,shapefile,False)
            temp_grid, maxmin = OKriging(latlon_dict,temp,hourly,var_name,shapefile,False)
            rh_grid, maxmin = OKriging(latlon_dict,rh,hourly,var_name,shapefile,False)
            wind_grid, maxmin = OKriging(latlon_dict,wind,hourly,var_name,shapefile,False)

        if (interpolation_method == 'OK' or interpolation_method == 'TPSS' or interpolation_method == 'TPS' or interpolation_method == 'IDEW-2'\
           or interpolation_method == 'IDEW-1' or interpolation_method == 'IDW-2' or interpolation_method == 'IDW-1') != True:

            print('The entered interpolation method is not recognized')
            sys.exit() 
        
        if count > 0:  
            dc_array = dc_list[count-1] #the last one added will be yesterday's val, but there's a lag bc none was added when count was0, so just use count-1
            index = count-1
            dc = DC(dat,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,dc_array,index,False,shapefile,mask1,endMask,last_DC_val_before_shutdown,overwinter)
            dc_list.append(dc)
        else: #Initialize procedure
            if overwinter:
                rain_shape = rain_grid.shape
                dc_initialize = np.zeros(rain_shape)
                dc_initialize[np.isnan(last_DC_val_before_shutdown)] = 15
                dc_initialize[~np.isnan(last_DC_val_before_shutdown)] = last_DC_val_before_shutdown[~np.isnan(last_DC_val_before_shutdown)]
                dc_list.append(dc_initialize*mask1)
            else: 
                rain_shape = rain_grid.shape
                dc_initialize = np.zeros(rain_shape)+15 #merge with the other overwinter array once it's calculated 
                dc_yesterday1 = dc_initialize*mask1
                dc_list.append(dc_yesterday1) #placeholder 
        count += 1

    if json:
        dc_list = [i.tolist() for i in dc_list]

    return dc_list


In [43]:
def get_last_dc_before_shutdown(dc_list,endsurface,overwinter_reqd,show,maxmin):
    '''Get an array of the last dc vals before station shutdown for winter for the study area 
    Parameters
        dc_list (list of np_array): a list of the interpolated surfaces for the drought code for 
        each day in the fire season 
        endsurface (np_array): array for end dates for the year before the year of interest 
        overwinter_reqd (np_array): where the overwinter procedure is required 
        show (bool): whether you want to plot the map 
        maxmin (list): bounds of the study area 
    Returns 
        last_DC_val_before_shutdown_masked_reshape (np_array): array of dc values before shutdown for 
        cells requiring overwinter procedure 
    '''

    flatten_dc = list(map(lambda x:x.flatten(),dc_list)) #flatten the arrays for easier processing - avoid 3d array
    stackDC = np.stack(flatten_dc,axis=-1) #Create an array from the list that we can index

    days_since_mar1 = endsurface.flatten().astype(int)+214-1 #add 214 days (mar-aug31 to convert to days since March 1) to get index in the stack... based on make_end_date_mask() -1 for day before

    last_DC_val_before_shutdown = stackDC[np.arange(len(stackDC)), days_since_mar1] #Index each cell in the array by the end date to get the last val
    last_DC_val_before_shutdown_masked = last_DC_val_before_shutdown * overwinter_reqd.flatten() #Mask the areas that don't require the overwinter procedure
    last_DC_val_before_shutdown_masked_reshape = last_DC_val_before_shutdown_masked.reshape(endsurface.shape)

    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(last_DC_val_before_shutdown_masked_reshape,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('DC') 
        
        title = 'Last DC'
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()
        
    return last_DC_val_before_shutdown_masked_reshape

In [44]:
def dc_overwinter_procedure(last_DC_val_before_shutdown_masked_reshape,overwinter_pcp,b_list):
    '''Apply the overwinter procedure, see Wang et al. (2017) for more details 
    Parameters
        last_DC_val_before_shutdown_masked_reshape (np_array): output of get_last_dc_before_shutdown, array containing the last
        dc value before the station shut down interpolated across the study area 
        overwinter_pcp (np_array): overwinter precipitation amount in each cell 
        b_list (list, loaded from json): list containing information for the b parameter, can be formatted into an array 
    Returns 
        DCs (np_array): the new values for the start-up DC procedure, to be used in areas identified as needing the overwinter
        procedure 
    '''
    a = 1.0
    b_array = np.array(b_list).reshape(overwinter_pcp.shape)
    Qf = 800 * np.exp(-last_DC_val_before_shutdown_masked_reshape / 400)
    Qs = a * Qf + b_array * (3.94 * overwinter_pcp)
    DCs = 400 * np.log(800 / Qs) #this is natural logarithm 

    DCs[DCs < 15] = 15
    return DCs

In [45]:
%%cython

import numpy as np
import os, sys
import matplotlib as plt 
import geopandas as gpd
from descartes import PolygonPatch 

def DMC(input_date,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,dmc_yesterday,int index,bint show,str shapefile,
        mask,endMask):
    '''Calculate the DMC. See cffdrs R code
    Parameters
        input_date (str): input date of interest
        rain_grid: interpolated surface for rainfall on the date of interest
        rh_grid: interpolated surface for relative humidity on the date of interest
        temp_grid: interpolated surface for temperature on the date of interest
        wind_grid: interpolated surface for wind on the date of interest
        maxmin: bounds of the study area 
        dmc_yesterday: array of DMC values for yesterday (from the dmc stack list/if this function
        is being used inside dmc_stack it is calculated then) 
        index (int): index of the date since Mar 1
        show (bool): whether you want to show the map 
        shapefile (str): path to the study area shapefile
        mask (np_array): mask for the start dates 
        endMask (np_array): mask for the end days 
    Returns 
        dmc (np_array): array of dmc values on the date on interest for the study area
    '''
    yesterday_index = index-1

    if yesterday_index == -1:
        rain_shape = rain_grid.shape
        dmc_initialize = np.zeros(rain_shape)+6
        dmc_yesterday1 = dmc_initialize*mask
    else: 
        dmc_yesterday1 = dmc_yesterday
        dmc_yesterday1[np.where(np.isnan(dmc_yesterday1) & ~np.isnan(mask))] = 6

    #dmc_yesterday = dmc_yesterday1.flatten()
    input_date = str(input_date)
    month = int(input_date[6])
    #Get day length factor

    ell01 = [6.5, 7.5, 9, 12.8, 13.9, 13.9, 12.4, 10.9, 9.4, 8, 7, 6]

    #Put constraint on low end of temp
    temp_grid[temp_grid < -1.1] = -1.1

    #Log drying rate
    rk = 1.84*(temp_grid+1.1)*(100-rh_grid)*ell01[month]*1.0E-4

    #Make empty dmc array
    new_shape = dmc_yesterday1.shape
    dmc = np.zeros(new_shape)

    #starting rain

    netRain = 0.92*rain_grid-1.27

    #initial moisture content, modified same as cffdrs package
    wmi = 20 + 280/np.exp(0.023*dmc_yesterday1)

    #if else depending on yesterday dmc, eq.13
    b = np.zeros(new_shape)


    b[dmc_yesterday1 <= 33] = 100/(0.5+0.3*dmc_yesterday1[dmc_yesterday1 <= 33])
    b[(dmc_yesterday1 > 33) & (dmc_yesterday1 < 65)] = 14-1.3*np.log(dmc_yesterday1[(dmc_yesterday1 > 33) & (dmc_yesterday1 < 65)]) # np.log is ln
    b[dmc_yesterday1 >= 65] = 6.5*np.log(dmc_yesterday1[dmc_yesterday1 >= 65])-17.2
        

    #eq 14, modified in R package
    wmr = wmi + 1000 * netRain/(48.77 + b * netRain)

    #eq 15 modified to be same as cffdrs package
    
    pr0 = 43.43 * (5.6348 - np.log(wmr-20)) #natural logarithm

    pr0[pr0 <0] = 0
    

    rk_pr0 =pr0 + rk
    rk_ydmc = dmc_yesterday1 + rk #we want to add rk because that's the drying rate 
    dmc[netRain > 1.5] = rk_pr0[netRain > 1.5]
    dmc[netRain <= 1.5] = rk_ydmc[netRain <= 1.5]



    dmc[dmc < 0]=0

    dmc = dmc * mask * endMask # mask out areas that haven't been activated



    if show == True: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
        circ = PolygonPatch(na_map['geometry'][0],visible=False)
        ax.add_patch(circ) 
        plt.imshow(dmc,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1),clip_path=circ, 
                   clip_on=True) 
        
      
        #plt.imshow(dmc,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('DMC') 
        
        title = 'DMC for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()

    return dmc 

In [46]:
def dmc_stack(dates,file_path_daily,file_path_hourly,var_name,shapefile,day_interpolated_surface,
end_interpolated_surface,file_path_elev,idx_list,date_dictionary,latlon_dict,latlon_dictionary,json,interpolation_method):
    '''Calc dmc for each day in fire season. For notes see cffdrs R code.
    Parameters
        dates (list): list of all dates within the fire season, inactive stations will be masked out 
        so you can define it as Mar 1 - Dec 31
        file_path_daily (str): file path to the daily feather files
        file_path_hourly (str): file path to the hourly feather files
        var_name (str): name of the variable you are interpolating
        shapefile (str): path to the study area shapefile 
        day_interpolated_surface (np_array): array of start-up days (since Mar 1) for the study area
        end_interpolated_surface (np_array): array of end days (since Oct 1) for the study area
        file_path_elev (str): file path to the elevation lookup file 
        idx_list (list): the index of the elevation data column in the lookup file 
        date_dictionary (dict, loaded from .json): lookup file that has what day/month pairs each 
        station contains data for 
        latlon_dict (dict, loaded from .json): lat lons of the hourly stations
        latlon_dictionary (dict, loaded from .json): lat lons of the daily stations
        json (bool): if True, convert the array to a flat list so it can be written as a .json file
        interpolation_method (str): the interpolation method to use to get the continuous DMC surface, 
        there are seven options - IDW-1, IDW-2, IDEW-1, IDEW-2, TPS, TPSS, OK 
    Returns 
        dmc_list (list of np_array): a list of the interpolated surfaces for the duff moisture code for 
        each day in the fire season 
    '''
    dmc_list = [] 
    count = 0 
    for dat in dates:
        index = dates.index(dat) #need to run BEFORE we convert to string 
        gc.collect() 
        year = str(dat)[0:4]
        dat = str(dat)
        day_index= get_date_index(year,dat,3)
        eDay_index = get_date_index(year,dat,10)

        mask = make_start_date_mask(day_index,day_interpolated_surface)
        if eDay_index < 0:
            endMask = np.ones(end_interpolated_surface.shape) #in the case that the index is before Oct 1
        else: 
            endMask = make_end_date_mask(eDay_index,end_interpolated_surface)
    
        hourly = str(dat)[0:10]+' 13:00'
        
        rainfall = get_pcp(str(dat)[0:10],file_path_daily,date_dictionary)
        wind = get_wind_speed(hourly,file_path_hourly) #Using the list, get the data for wind speed for those stations on the input date
        temp = get_noon_temp(hourly,file_path_hourly) #Using the list, get the data for temperature for those stations on the input date
        rh = get_relative_humidity(hourly,file_path_hourly) #Using the list, get the data for rh% for those stations on the input date
        
        #what type of interpolation are we using here?

        if interpolation_method == 'IDW-1': 

        
            rain_grid, maxmin = IDW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,1)
            temp_grid, maxmin = IDW(latlon_dict,temp,hourly,var_name,shapefile,False,1)
            rh_grid, maxmin = IDW(latlon_dict,rh,hourly,var_name,shapefile,False,1)
            wind_grid, maxmin = IDW(latlon_dict,wind,hourly,var_name,shapefile,False,1)

        if interpolation_method == 'IDW-2': 

        
            rain_grid, maxmin = IDW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,2)
            temp_grid, maxmin = IDW(latlon_dict,temp,hourly,var_name,shapefile,False,2)
            rh_grid, maxmin = IDW(latlon_dict,rh,hourly,var_name,shapefile,False,2)
            wind_grid, maxmin = IDW(latlon_dict,wind,hourly,var_name,shapefile,False,2)

        if interpolation_method == 'IDEW-1':

            rain_grid, maxmin, elev_array  = IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,1)
            temp_grid, maxmin, elev_array  = IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            rh_grid, maxmin, elev_array  = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            wind_grid, maxmin, elev_array  = IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            
        if interpolation_method == 'IDEW-2':

            rain_grid, maxmin, elev_array  = IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,2)
            temp_grid, maxmin, elev_array  = IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            rh_grid, maxmin, elev_array  = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            wind_grid, maxmin, elev_array  = IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)

        if interpolation_method == 'TPS':

            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,0)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,0)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,0)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,0)

        if interpolation_method == 'TPSS':

            num_stations_R = len(rainfall.keys())
            num_stations_t = len(temp.keys())
            num_stations_rh = len(rh.keys())
            num_stations_w = len(wind.keys())
            
            smoothing_parameterR = int(num_stations_R)-(math.sqrt(2*num_stations_R))
            smoothing_parameterT = int(num_stations_t)-(math.sqrt(2*num_stations_t))
            smoothing_parameterRH = int(num_stations_rh)-(math.sqrt(2*num_stations_rh))
            smoothing_parameterW = int(num_stations_w)-(math.sqrt(2*num_stations_w))
            
            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,smoothing_parameterR)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,smoothing_parameterT)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,smoothing_parameterRH)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,smoothing_parameterW)


        if interpolation_method == 'OK':

            rain_grid, maxmin = OKriging(latlon_dictionary,rainfall,dat,var_name,shapefile,False)
            temp_grid, maxmin = OKriging(latlon_dict,temp,hourly,var_name,shapefile,False)
            rh_grid, maxmin = OKriging(latlon_dict,rh,hourly,var_name,shapefile,False)
            wind_grid, maxmin = OKriging(latlon_dict,wind,hourly,var_name,shapefile,False)

        if (interpolation_method == 'OK' or interpolation_method == 'TPSS' or interpolation_method == 'TPS' or interpolation_method == 'IDEW-2'\
           or interpolation_method == 'IDEW-1' or interpolation_method == 'IDW-2' or interpolation_method == 'IDW-1') != True:

            print('The entered interpolation method is not recognized')
            sys.exit()
            
        if count > 0:  
            dmc_array = dmc_list[count-1] #the last one added will be yesterday's val, but there's a lag bc none was added when count was0, so just use count-1
            index = count-1
            dmc = DMC(dat,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,dmc_array,index,False,shapefile,mask,endMask)
            dmc_list.append(dmc)
        else:
            rain_shape = rain_grid.shape
            dmc_initialize = np.zeros(rain_shape)+6
            dmc_yesterday1 = dmc_initialize*mask
            dmc_list.append(dmc_yesterday1) #placeholder 
        count += 1

    if json:
        dmc_list = [i.tolist() for i in dmc_list]

    return dmc_list

In [47]:
%%cython

import numpy as np
import os, sys
import matplotlib as plt 
import geopandas as gpd
from descartes import PolygonPatch 

def FFMC(input_date,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,ffmc_yesterday,int index,bint show,str shapefile,
         mask,endMask):
    '''Calculate the FFMC. See cffdrs R code
    Parameters
        input_date (str): input date of interest
        rain_grid: interpolated surface for rainfall on the date of interest
        rh_grid: interpolated surface for relative humidity on the date of interest
        temp_grid: interpolated surface for temperature on the date of interest
        wind_grid: interpolated surface for wind on the date of interest
        maxmin: bounds of the study area 
        ffmc_yesterday: array of FFMC values for yesterday (from the ffmc stack list/if this function
        is being used inside ffmc_stack it is calculated then) 
        index (int): index of the date since Mar 1
        show (bool): whether you want to show the map 
        shapefile (str): path to the study area shapefile
        mask (np_array): mask for the start dates 
        endMask (np_array): mask for the end days 
    Returns 
        ffmc1 (np_array): array of ffmc values on the date on interest for the study area
    '''
    cdef int yesterday_index = index-1

    if yesterday_index == -1:
        rain_shape = rain_grid.shape
        ffmc_initialize = np.zeros(rain_shape)+85
        ffmc_yesterday1 = ffmc_initialize*mask #mask out areas that haven't started
    else: 
        ffmc_yesterday1 = ffmc_yesterday
        ffmc_yesterday1[np.where(np.isnan(ffmc_yesterday1) & ~np.isnan(mask))] = 85 #set started pixels since yesterday to 85


    wmo = 147.2*(101-ffmc_yesterday)/(59.5+ffmc_yesterday)

    rain_grid[rain_grid > 0.5] = rain_grid[rain_grid > 0.5] - 0.5

    wmo[wmo>=150]=wmo[wmo >= 150]+0.0015*(wmo[wmo >= 150]-150)*\
                     (wmo[wmo >= 150]- 150)*np.sqrt(rain_grid[wmo >= 150]) + 42.5\
                     *rain_grid[wmo >= 150]*np.exp(-100/(251-wmo[wmo >= 150]))*\
                     (1-np.exp(-6.93/rain_grid[wmo >= 150]))
    
    wmo[wmo<150]=wmo[wmo<150]+42.5*rain_grid[wmo<150]*np.exp(-100/(251-wmo[wmo<150]))\
                  *(1-np.exp(-6.93/rain_grid[wmo<150]))

    wmo[rain_grid < 0.5] = 147.2*(101-ffmc_yesterday[rain_grid < 0.5])/(59.5+ffmc_yesterday[rain_grid < 0.5])

    wmo[wmo>250] = 250

    ed=0.942*np.power(rh_grid,0.679)+(11*np.exp((rh_grid-100)/10))+0.18*(21.1-temp_grid)\
        *(1-1/np.exp(rh_grid*0.115))
    
    ew=0.618*np.power(rh_grid,0.753)+(10*np.exp((rh_grid-100)/10))+0.18*(21.1-temp_grid)*\
        (1-1/np.exp(rh_grid*0.115))

    shape = rain_grid.shape 
    z = np.zeros(shape)
    z[np.where((wmo<ed) & (wmo<ew))]=0.424*(1-np.power((rh_grid[np.where((wmo<ed) & (wmo<ew))]/100),1.7))\
                          +0.0694*np.sqrt(wind_grid[np.where((wmo<ed) & (wmo<ew))])*\
                  (1-np.power((rh_grid[np.where((wmo<ed) & (wmo<ew))]/100),8))
    

    z[np.where((wmo>=ed) & (wmo>=ew))] = 0

    x=z*0.581*np.exp(0.0365*temp_grid)

    shape = rain_grid.shape 
    wm = np.zeros(shape)    

    wm[np.where((wmo<ed) & (wmo<ew))]= ew[np.where((wmo<ed) & (wmo<ew))]-\
                           (ew[np.where((wmo<ed) & (wmo<ew))]-\
                            wmo[np.where((wmo<ed) & (wmo<ew))])/(np.power(10,x[np.where((wmo<ed) & (wmo<ew))]))

    wm[np.where((wmo>=ed) & (wmo>=ew))] = wmo[np.where((wmo>=ed) & (wmo>=ew))]

    z[wmo>ed] = 0.424*(1-np.power((rh_grid[wmo>ed]/100),1.7))+0.0694\
                       *np.sqrt(wind_grid[wmo>ed])*(1-np.power((rh_grid[wmo>ed]/100),8))

    x=z*0.581*np.exp(0.0365 * temp_grid)
    wm[wmo>ed] = ed[wmo>ed] + (wmo[wmo>ed] - ed[wmo>ed])/(np.power(10,x[wmo>ed]))

    ffmc1 = (59.5*(250-wm))/(147.2+wm)
    
    ffmc1[ffmc1 > 101] = 101

    ffmc1[ffmc1 < 0] = 0

    ffmc1 = ffmc1*mask* endMask

    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
        circ = PolygonPatch(na_map['geometry'][0],visible=False)
        ax.add_patch(circ) 
        plt.imshow(ffmc1,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1),clip_path=circ, 
                   clip_on=True) 
        
        #plt.imshow(ffmc1,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('FFMC') 
        
        title = 'FFMC for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()

    return ffmc1

In [48]:
def ffmc_stack(dates,file_path_daily,file_path_hourly,var_name,shapefile,day_interpolated_surface,
               end_interpolated_surface,file_path_elev,idx_list,date_dictionary,latlon_dict,latlon_dictionary,
               json,interpolation_method):
    '''Calc ffmc for each day in season. For notes see cffdrs R code.
    Parameters
        dates (list): list of all dates within the fire season, inactive stations will be masked out 
        so you can define it as Mar 1 - Dec 31
        file_path_daily (str): file path to the daily feather files
        file_path_hourly (str): file path to the hourly feather files
        var_name (str): name of the variable you are interpolating
        shapefile (str): path to the study area shapefile 
        day_interpolated_surface (np_array): array of start-up days (since Mar 1) for the study area
        end_interpolated_surface (np_array): array of end days (since Oct 1) for the study area
        file_path_elev (str): file path to the elevation lookup file 
        idx_list (list): the index of the elevation data column in the lookup file 
        date_dictionary (dict, loaded from .json): lookup file that has what day/month pairs each 
        station contains data for 
        latlon_dict (dict, loaded from .json): lat lons of the hourly stations
        latlon_dictionary (dict, loaded from .json): lat lons of the daily stations
        json (bool): if True, convert the array to a flat list so it can be written as a .json file
        interpolation_method (str): the interpolation method to use to get the continuous DMC surface, 
        there are seven options - IDW-1, IDW-2, IDEW-1, IDEW-2, TPS, TPSS, OK 
    Returns 
        ffmc_list (list of np_array): a list of the interpolated surfaces for the fine fuel moisture code for 
        each day in the fire season 
    '''
    ffmc_list = [] 
    count = 0 
    for dat in dates:
        index = dates.index(dat) #need to run BEFORE we convert to string 
        gc.collect() 
        year = str(dat)[0:4]
        dat = str(dat) #convert to str so that cython doesn't get confused 
        day_index= get_date_index(year,dat,3)
        eDay_index = get_date_index(year,dat,10)

        mask = make_start_date_mask(day_index,day_interpolated_surface)
        if eDay_index < 0:
            endMask = np.ones(end_interpolated_surface.shape) #in the case that the index is before Oct 1
        else: 
            endMask = make_end_date_mask(eDay_index,end_interpolated_surface)

        hourly = str(dat)[0:10]+' 13:00'
        rainfall = get_pcp(str(dat)[0:10],file_path_daily,date_dictionary)
        wind = get_wind_speed(hourly,file_path_hourly) #Using the list, get the data for wind speed for those stations on the input date
        temp = get_noon_temp(hourly,file_path_hourly) #Using the list, get the data for temperature for those stations on the input date
        rh = get_relative_humidity(hourly,file_path_hourly) #Using the list, get the data for rh% for those stations on the input date
        
        #what type of interpolation are we using here?

        if interpolation_method == 'IDW-1': 

        
            rain_grid, maxmin = IDW(latlon_dictionary,rainfall,str(dat),var_name,shapefile,False,1)
            temp_grid, maxmin = IDW(latlon_dict,temp,hourly,var_name,shapefile,False,1)
            rh_grid, maxmin = IDW(latlon_dict,rh,hourly,var_name,shapefile,False,1)
            wind_grid, maxmin = IDW(latlon_dict,wind,hourly,var_name,shapefile,False,1)

        if interpolation_method == 'IDW-2': 

        
            rain_grid, maxmin = IDW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,2)
            temp_grid, maxmin = IDW(latlon_dict,temp,hourly,var_name,shapefile,False,2)
            rh_grid, maxmin = IDW(latlon_dict,rh,hourly,var_name,shapefile,False,2)
            wind_grid, maxmin = IDW(latlon_dict,wind,hourly,var_name,shapefile,False,2)

        if interpolation_method == 'IDEW-1':

            rain_grid, maxmin, elev_array = IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,1)
            temp_grid, maxmin, elev_array = IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            rh_grid, maxmin, elev_array = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            wind_grid, maxmin, elev_array = IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,1)
            
        if interpolation_method == 'IDEW-2':

            rain_grid, maxmin, elev_array = IDEW(latlon_dictionary,rainfall,dat,var_name,shapefile,False,file_path_elev,idx_list,2)
            temp_grid, maxmin, elev_array = IDEW(latlon_dict,temp,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            rh_grid, maxmin, elev_array = IDEW(latlon_dict,rh,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)
            wind_grid, maxmin, elev_array = IDEW(latlon_dict,wind,hourly,var_name,shapefile,False,file_path_elev,idx_list,2)

        if interpolation_method == 'TPS':

            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,0)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,0)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,0)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,0)

        if interpolation_method == 'TPSS':

            num_stations_R = len(rainfall.keys())
            num_stations_t = len(temp.keys())
            num_stations_rh = len(rh.keys())
            num_stations_w = len(wind.keys())
            
            smoothing_parameterR = int(num_stations_R)-(math.sqrt(2*num_stations_R))
            smoothing_parameterT = int(num_stations_t)-(math.sqrt(2*num_stations_t))
            smoothing_parameterRH = int(num_stations_rh)-(math.sqrt(2*num_stations_rh))
            smoothing_parameterW = int(num_stations_w)-(math.sqrt(2*num_stations_w))
            
            rain_grid, maxmin = TPS(latlon_dictionary,rainfall,dat,var_name,shapefile,False,smoothing_parameterR)
            temp_grid, maxmin = TPS(latlon_dict,temp,hourly,var_name,shapefile,False,smoothing_parameterT)
            rh_grid, maxmin = TPS(latlon_dict,rh,hourly,var_name,shapefile,False,smoothing_parameterRH)
            wind_grid, maxmin = TPS(latlon_dict,wind,hourly,var_name,shapefile,False,smoothing_parameterW)

        if interpolation_method == 'OK':

            rain_grid, maxmin = OKriging(latlon_dictionary,rainfall,dat,var_name,shapefile,False)
            temp_grid, maxmin = OKriging(latlon_dict,temp,hourly,var_name,shapefile,False)
            rh_grid, maxmin = OKriging(latlon_dict,rh,hourly,var_name,shapefile,False)
            wind_grid, maxmin = OKriging(latlon_dict,wind,hourly,var_name,shapefile,False)

        if (interpolation_method == 'OK' or interpolation_method == 'TPSS' or interpolation_method == 'TPS' or interpolation_method == 'IDEW-2'\
           or interpolation_method == 'IDEW-1' or interpolation_method == 'IDW-2' or interpolation_method == 'IDW-1') != True:

            print('The entered interpolation method is not recognized')
            sys.exit()
            
        if count > 0:  
            ffmc_array = ffmc_list[count-1] #the last one added will be yesterday's val, but there's a lag bc none was added when count was0, so just use count-1
            index = count-1
            ffmc = FFMC(dat,rain_grid,rh_grid,temp_grid,wind_grid,maxmin,ffmc_array,index,False,shapefile,mask,endMask)
            ffmc_list.append(ffmc)
        else:
            rain_shape = rain_grid.shape
            ffmc_initialize = np.zeros(rain_shape)+85
            ffmc_yesterday1 = ffmc_initialize*mask
            ffmc_list.append(ffmc_yesterday1) #placeholder
                
                
        count += 1

    if json:
        ffmc_list = [i.tolist() for i in ffmc_list]

    return ffmc_list

In [ ]:
def BUI(dmc,dc,maxmin,show,shapefile,mask,endMask): #BUI can be calculated on the fly
    ''' Calculate BUI
    Parameters
        dmc (np_array): the dmc array for the date of interest
        dc (np_array): the dc array for the date of interest
        maxmin (list): bounds of the study area 
        show (bool): whether or not to display the map 
        shapefile (str): path to the study area shapefile 
        mask (np_array): mask for the start up date 
        endMask (np_array): mask for the shut down date 
    Returns 
        bui1 (np_array): array containing BUI values for the study area 
    '''
    shape = dmc.shape
    bui1 = np.zeros(shape)
    
    bui1[np.where((dmc == 0) & (dc == 0))] = 0
    bui1[np.where((dmc > 0) & (dc > 0))] = 0.8 * dc[np.where((dmc > 0) & (dc > 0))]*\
                                          dmc[np.where((dmc > 0) & (dc > 0))]/(dmc[np.where((dmc > 0) & (dc > 0))]\
                                          + 0.4 * dc[np.where((dmc > 0) & (dc > 0))])
    p = np.zeros(shape)
    p[dmc == 0] = 0
    p[dmc > 0] = (dmc[dmc > 0] - bui1[dmc > 0])/dmc[dmc > 0]

    cc = 0.92 + (np.power((0.0114 * dmc),1.7))
    
    bui0 = dmc - cc * p

    bui0[bui0 < 0] = 0

    bui1[bui1 < dmc] = bui0[bui1 < dmc]

    bui1 = bui1*mask* endMask

    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(bui1,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('BUI') 
        
        title = 'BUI for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()

    return bui1

In [63]:
import matplotlib.pyplot as plt

In [65]:
def ISI(ffmc,wind_grid,maxmin,show,shapefile,mask,endMask):
    ''' Calculate ISI
    Parameters
        ffmc (np_array): ffmc array for the date of interest
        wind_grid (np_array): wind speed interpolated array for the date of interest
        maxmin (list): bounds of the shapefile
        show (bool): whether or not to display the map 
        shapefile (str): path to the study area shapefile
        mask (np_array): mask for the start up date
        endMask (np_array): mask for the shutdown date
    Returns 
        isi (np_array): the calculated array for ISI for the study area 
    '''

    fm = 147.2 * (101 - ffmc)/(59.5 + ffmc)

    
    fW = np.exp(0.05039 * wind_grid)
    

    fF = 91.9 * np.exp((-0.1386) * fm) * (1 + (fm**5.31) / 49300000)


    isi = 0.208 * fW * fF
    
    isi = isi*mask* endMask
    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(isi,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('ISI') 
        
        title = 'ISI for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()
        
    return isi

In [67]:
def FWI(isi,bui,maxmin,show,shapefile,mask,endMask):
    ''' Calculate FWI
    Parameters
        isi (np_array): calculated isi surface for the date of interest 
        bui (np_array): calculated bui surface for the date of interest 
        maxmin (list): bounds of the study area
        show (bool): whether or not to show the map 
        shapefile (str): path to the shapefile 
        mask (np_array): start up mask 
        endMask (np_array): shut down mask 
    Returns 
        fwi (np_array): calculated FWI surface for the study area 
    '''

    shape = isi.shape
    bb = np.zeros(shape)

    bb[bui > 80] = 0.1 * isi[bui > 80] * (1000/(25 + 108.64/np.exp(0.023 * bui[bui > 80])))
    bb[bui <= 80] =  0.1 * isi[bui <= 80] * (0.626 * np.power(bui[bui <= 80],0.809) + 2)

    fwi = np.zeros(shape)
    fwi[bb <= 1] = bb[bb <= 1]
    fwi[bb > 1] = np.exp(2.72 * ((0.434 * np.log(bb[bb > 1]))**0.647)) #natural logarithm 

    fwi = fwi * mask * endMask

    if show: 
        min_yProj_extent = maxmin[0]
        max_yProj_extent = maxmin[1]
        max_xProj_extent = maxmin[2]
        min_xProj_extent = maxmin[3]

        fig, ax = plt.subplots(figsize= (15,15))
        crs = {'init': 'esri:102001'}

        na_map = gpd.read_file(shapefile)
        
      
        plt.imshow(fwi,extent=(min_xProj_extent-1,max_xProj_extent+1,max_yProj_extent-1,min_yProj_extent+1)) 
        na_map.plot(ax = ax,color='white',edgecolor='k',linewidth=2,zorder=10,alpha=0.1)
            
        plt.gca().invert_yaxis()
        cbar = plt.colorbar()
        cbar.set_label('FWI') 
        
        title = 'FWI for %s'%(input_date) 
        fig.suptitle(title, fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        
        plt.show()
 
    return fwi

The next task to complete is to find the highest FWI metrics inside of an individual fire during the first four days since the report date. See: 

Amiro, B. D., Logan, K. A., Wotton, B. M., Flannigan, M. D., Todd, J. B., Stocks, B. J., & Martell, D. L. (2004). Fire weather index system components for large fires in the Canadian boreal forest. International Journal of Wildland Fire, 13(4), 391–400. https://doi.org/10.1071/WF03066

In [69]:
from shapely.geometry import Point
from scipy.spatial.distance import cdist

In [70]:
def get_interpolated_val_in_fire(fire_shapefile,shapefile,latlon_dict,interpolated_surface):
    '''This is a function to get the FWI metric value inside the fire.
    We will use to calculate the max FWI metrics for a fire.
    Parameters
        fire_shapefile (str): path to the fire shapefile 
        shapefile (str): path to the study area shapefile
        latlon_dict (dict, loaded from json): dictionary of lat lon for each station 
        interpolated_surface (np_array): an array of values in the study area 
    Returns 
        ival, max_ival (either, float): maximum value in fire, either the closest point to the convex hull of the fire or 
        a sum of the points inside the fire
    '''
    lat = [] #Initialize empty lists to store data 
    lon = []

    for station_name in latlon_dict.keys(): #Loop through the list of stations 
        loc = latlon_dict[station_name]
        latitude = loc[0]
        longitude = loc[1]
        lat.append(float(latitude))
        lon.append(float(longitude))

    y = np.array(lat) #Convert to a numpy array for faster processing speed 
    x = np.array(lon)


    na_map = gpd.read_file(shapefile)
    bounds = na_map.bounds #Get the bounding box of the shapefile 
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']
    pixelHeight = 10000 #We want a 10 by 10 pixel, or as close as we can get 
    pixelWidth = 10000
            
    num_col = int((xmax - xmin) / pixelHeight) #Calculate the number of rows cols to fill the bounding box at that resolution 
    num_row = int((ymax - ymin) / pixelWidth)

    source_proj = pyproj.Proj(proj='latlong', datum = 'NAD83') #We dont know but assume NAD83
    xProj, yProj = pyproj.Proj('esri:102001')(x,y) #Convert to Canada Albers Equal Area 

    yProj_extent=np.append(yProj,[bounds['maxy'],bounds['miny']]) #Add the bounding box coords to the dataset so we can extrapolate the interpolation to cover whole area
    xProj_extent=np.append(xProj,[bounds['maxx'],bounds['minx']])
    

    Yi = np.linspace(np.min(yProj_extent),np.max(yProj_extent),num_row) #Get the value for lat lon in each cell we just made 
    Xi = np.linspace(np.min(xProj_extent),np.max(xProj_extent),num_col)

    Xi,Yi = np.meshgrid(Xi,Yi)
    concat = np.array((Xi.flatten(), Yi.flatten())).T #Because we are not using the lookup file, send in X,Y order 
    send_to_list = concat.tolist()

    fire_map = gpd.read_file(fire_shapefile)
    DF = fire_map.geometry.unary_union

    meshPoints = [Point(item) for item in send_to_list]
    df = pd.DataFrame(meshPoints)
    gdf = gpd.GeoDataFrame(df, geometry=meshPoints)
    
    within_fire = gdf[gdf.geometry.within(DF)]


    if len(within_fire) == 0:
        #Get concave hull of the multipolygon
        try: 
            bounding_box = DF.convex_hull
            approx_centre_point = bounding_box.centroid.coords
        except: #theres only one polygon
            approx_centre_point = DF.centroid.coords

        reshaped_coord= np.array(approx_centre_point)
        hypot = cdist(concat,reshaped_coord)
        where_distance_small = np.argmin(hypot) #argmin returns the index of where the smallest item is 

        centre = concat[where_distance_small]

        #Get interpolated value here
        intF = interpolated_surface.flatten() 
        ival = intF[where_distance_small]
 

        return ival 
    
    else: 

        intF = interpolated_surface.flatten()
        listP = within_fire[0].tolist()
        tupArray =[x.coords for x in listP]
        xyFire = [(x[0][0],x[0][1],) for x in tupArray]

        tup_sendtolist =[tuple(x) for x in send_to_list]
        
        index = [] 
        for pair in xyFire:

        
            indices = [i for i, x in enumerate(tup_sendtolist) if x == pair]

            index.append(indices)

        ivals = []
        for i in index: 

            ivals.append(intF[i])


        max_ival = max(ivals) #get the max val inside the fire

        
        return float(max_ival)

In [71]:
def highest_value_first_four_days(fire_shapefile,shapefile,latlon_dict,interpolated_surface_d1,interpolated_surface_d2,
                                  interpolated_surface_d3,interpolated_surface_d4):
    '''Function to return the highest FWI values in a fire for four input arrays  
    Parameters
        fire_shapefile (str): path to the fire shapefile 
        shapefile (str): path to the study area shapefile
        latlon_dict (dict, loaded from json): dictionary of lat lon for each station 
        interpolated_surface_d1,d2,d3,d4 (np_array): an array of values in the study area, in the first four days of the fire
    Returns 
        max_val (float): maximum value for first four days since report date 
    '''
    v1 = get_interpolated_val_in_fire(fire_shapefile,shapefile,latlon_dict,interpolated_surface_d1)
    v2 = get_interpolated_val_in_fire(fire_shapefile,shapefile,latlon_dict,interpolated_surface_d2)
    v3 = get_interpolated_val_in_fire(fire_shapefile,shapefile,latlon_dict,interpolated_surface_d3)
    v4 = get_interpolated_val_in_fire(fire_shapefile,shapefile,latlon_dict,interpolated_surface_d4)
    list_vals = [v1,v2,v3,v4]
    max_val = max(list_vals)
    return max_val

In [72]:
def get_report_date_plus_three(fire_shapefile):
    '''Function to return the first four days of the fire
    Parameters
        fire_shapefile (str): path to the fire shapefile 
    Returns 
        fire_dates (list): list of the report date and three days after
    '''
    fire_map = gpd.read_file(fire_shapefile)
    rep_date = pd.to_datetime(fire_map['REP_DATE'].to_list()[0])
    fire_dates = [str(rep_date)[0:10]] 
    for i in range(1,4):
        next_date = rep_date+pd.DateOffset(i)
        fire_dates.append(str(next_date)[0:10])

    return fire_dates 

Finally, we can evaluate the relationship of the FWI metrics with area burned using ridge regression. See: 
https://en.wikipedia.org/wiki/Tikhonov_regularization
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html

In [60]:
from sklearn import linear_model
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.preprocessing import QuantileTransformer

import math
import seaborn as sns 

In [61]:
def ridge_regression(path_to_excel_spreadsheet,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,all_variables,plot_distributions,
                     plot_residual_histogram,transform): 
    '''Make a ridge regression model and print out the resulting coefficients and (if True) the histogram of residuals 
    Parameters
        path_to_excel_spreadsheet (str): path to the spreadsheet containing the FWI values for each fire and the other covariates,
        Notes: No trailing 0s in speadsheet, no space after parameter name in header 
        var1-10 (str): variable names, corresponding to the header titles 
        all_variables (bool): if True, will use all the variables, not just the FWI metrics 
        plot_distributions (bool): if True, will plot the correlation diagram for all the variables 
        plot_residual_histogram (bool): if True, will plot a histogram showing the residuals to check if normally distributed
        transform (bool): if True, it will transform the input data (i.e. the fire surface area), 
        to make it normally distributed
    Returns 
        Prints out regression coefficients, MAE, and R2 of the model 
    '''

    df = pd.read_csv(path_to_excel_spreadsheet)
    mod = RidgeCV() 
    y = np.array(df['CALC_HA']).reshape(-1, 1)
    
    if all_variables: 
        X = np.array(df[[var1,var2,var3,var4,var5,var6,var7,var8,var9,var10]])
    else: 
        X = np.array(df[[var5,var6,var7,var8,var9,var10]]) #just fwi
    
    mod.fit(X,y)
    y_pred = mod.predict(X)
    
    if plot_distributions: 
        
        if all_variables: 

            dataset = df[['CALC_HA',var1,var2,var3,var4,var5,var6,var7,var8,var9,var10]]
        else: 
            dataset = df[['CALC_HA',var1,var2,var3,var4,var5,var6,var7,var8,var9,var10]]

        _ = sns.pairplot(dataset, kind='reg', diag_kind='kde') #check for correlation

        _.fig.set_size_inches(15,15)
    
    print('Coefficients: %s'%mod.coef_)
    print('Mean squared error: %s'% mean_absolute_error(df['CALC_HA'], y_pred))
    print('Coefficient of determination: %s'% r2_score(df['CALC_HA'], y_pred))

    f,  ax0 = plt.subplots(1, 1)
    ax0.scatter(df['CALC_HA'], y_pred)
    ax0.plot([0, 150000], [0, 150000], '--k')
    ax0.set_ylabel('Target predicted')
    ax0.set_xlabel('True Target')
    ax0.set_title('Ridge Regression \n without target transformation')
    ax0.set_xlim([0, 150000])
    ax0.set_ylim([0, 150000])
    plt.show()

    if plot_residual_histogram:
        fig, ax = plt.subplots()
        residuals = y - mod.predict(X)
        
        mu = sum(residuals)/len(residuals)
        var  = sum(pow(x-mu,2) for x in residuals) / len(residuals)
        sigma  = math.sqrt(var)
        n, bins, patches = ax.hist(residuals,50,density=1,align='left')
        line = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
        ax.plot(bins, line, '--')
        ax.set_xlabel('Residuals')
        ax.set_ylabel('Frequency')
        plt.show()

    if transform:
        mod2 = RidgeCV() 

        bc =QuantileTransformer(output_distribution='normal')
        y_trans_bc = bc.fit(y).transform(y)
        mod2.fit(X,y_trans_bc)

        y_pred2 = mod2.predict(X) 
        
        #Make new histogram
        fig, ax = plt.subplots()

        residuals = y - mod2.predict(X)
        
        mu = sum(residuals)/len(residuals)
        var  = sum(pow(x-mu,2) for x in residuals) / len(residuals)
        sigma  = math.sqrt(var)
        n, bins, patches = ax.hist(residuals,50,density=1,align='left')
        ax.set_xlabel('Residuals')
        ax.set_ylabel('Frequency')
        plt.show()
        
        print('Coefficients (T): %s'%mod.coef_)
        print('Mean squared error (T): %s'% mean_absolute_error(df['CALC_HA'], y_pred2))
        print('Coefficient of determination (T): %s'% r2_score(df['CALC_HA'], y_pred2))

        f, (ax0, ax1) = plt.subplots(1, 2, sharey=True)
        ax0.scatter(df['CALC_HA'], y_pred)
        ax0.plot([0, 150000], [0, 150000], '--k')
        ax0.set_ylabel('Target predicted')
        ax0.set_xlabel('True Target')
        ax0.set_title('Ridge Regression \n without transformation')
        ax0.set_xlim([0, 150000])
        ax0.set_ylim([0, 150000])

        ax1.scatter(df['CALC_HA'], y_pred2)
        ax1.plot([0, 150000], [0, 150000], '--k')
        ax1.set_ylabel('Target predicted')
        ax1.set_xlabel('True Target')
        ax1.set_title('Ridge Regression \n with transformation')
        ax1.set_xlim([0, 150000])
        ax1.set_ylim([0, 150000])

        f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])
        plt.show()
        